# BRAND_COLORS
- Voy a tratar de hacer un SUPER DICCIONAROTE con todos los brandas colors conocidos???
- Los podría separar por finanzas, Tech, Automotive (despues lo vemos por ahí se lo tiro a una IA para que lo haga)

## DESCARGAR de brandcolors.net el CSS o el archivo que sea
- Luego de ese archivo creo el diccionario...
- Entiendo que podría hacer la descarga con Web Scrapping pero por ahora lo vamos a hacer con es descargar el archivo en un formato de texto (ofrece CSS, SCC,... ) y tocarlo con código para que quede el DIC que me interesa a mí
- En vez de escribir el código que proceso lo bajado podría tirarselo a una IA pero vamos a proecesar el código con Python pa´codificar un poco
    - Esto lo puedo hacer con strings y lists de py o con re, vamos a empezar SIN re
    - de Source vamos a tomar el formato .styl que es el más simple

In [ ]:
src_file = "C:\\Users\\jm-mtm\\Downloads\\'brandcolors-1755614342.styl"

keys = []
values = []
# cnt = 0
with open(src_file, 'r') as f:         
     lines = f.readlines()
     for line in lines:
        #  if cnt > 4:
        #      break
        #  print(line, '-' * 25)
         if line:
             key, value = line.split('=', 1)
            #  key = key.split(':', 1)[1].strip()
             keys.append(key.split('-', 1)[1].strip())
             values.append(value.strip())
            #  cnt += 1

# print(keys)
# print(values)

BCs = dict(zip(keys, values))
BCs_s = dict(sorted(BCs.items()))

# print(BCs_s)
# BCs_s # voy a guardar el diccionario en un archivo porque la cell output NO me lo muestra todo . ni squiera el text ?? por? no sé


In [ ]:
import json
with open('brand_colors.json', 'w', encoding='utf-8') as f:
    json.dump(BCs_s, f, ensure_ascii=False, indent=4)

## Si quisiera leer el .json
with open('brand_colors.json', 'r', encoding='utf-8') as f:
    BCs_loaded = json.load(f)

BCs_loaded

## Trabajar los MINI-DIC a partir de lo que me dio Qewn
- Con el dic que me armo Qwen (los mini dics y voy a salvar 3 o 4 como ser Tech, Finance, Automotive, y Travel para guardar en mi modulo)

## NO SIRVE A esta altura ya armé dos JSON: (chatgpt)
- TENGO que MODIFICAR mucho el código para que me lo deje como quiero
- VOy a seguir arriba con el dic que me armo Qwen (los mini dics y voy a salvar 3 o 4 como ser Tech, Finance, Automotive, y Travel para guardar en mi modulo)
1. brand_colors.json (Estan TODOS los de brandcolors.net)
2. brand_cats.json (FALTAN brands)
    - Pero con un script que saca una resultante a no categorizada voy a poder determinar cuales faltam
    - Además en brand_cats NO están por orden alfabético (ya lo voy a arreglar.)

In [5]:
## Script que separa colors en distintos JSON según categoría
import json
import csv
from pathlib import Path

def main():
    input_file = Path("brand_colors.json")       # archivo original
    categories_file = Path("brand_cats.json")    # archivo de categorías definido por vos
    output_dir = Path("output_json")
    output_dir.mkdir(exist_ok=True)

    # Leer archivo de marcas
    with open(input_file, "r", encoding="utf-8") as f:
        brands = json.load(f)

    # Leer categorías
    with open(categories_file, "r", encoding="utf-8") as f:
        categories = json.load(f)

    # Normalizar: agrupar todos los colores de cada marca en una lista
    normalized = {}
    for brand, color in brands.items():
        base_name = brand.split("-")[0]  # "adobe-1" -> "adobe"
        normalized.setdefault(base_name, []).append(color)

    total_brands = len(normalized)

    # --- Alternativa 1: Un solo JSON con todas las categorías ---
    categorized = {cat: {} for cat in categories}
    categorized["Uncategorized"] = {}  # categoría especial

    # Asignar marcas a categorías
    all_categorized = set()
    for cat, brand_list in categories.items():
        for b in brand_list:
            if b in normalized:
                categorized[cat][b] = normalized[b]
                all_categorized.add(b)

    # Detectar marcas no categorizadas
    for b, colors in normalized.items():
        if b not in all_categorized:
            categorized["Uncategorized"][b] = colors

    # Guardar JSON combinado
    with open(output_dir / "brands_by_category.json", "w", encoding="utf-8") as f:
        json.dump(categorized, f, indent=4, ensure_ascii=False)

    # --- Alternativa 2: Un JSON por categoría ---
    for cat, brand_data in categorized.items():
        filename = "uncategorized.json" if cat == "Uncategorized" else f"{cat.lower()}.json"
        with open(output_dir / filename, "w", encoding="utf-8") as f:
            json.dump(brand_data, f, indent=4, ensure_ascii=False)

    # --- Resumen en consola ---
    print("\n📊 Resumen de categorización:")
    print(f"   Total de marcas detectadas: {total_brands}")
    for cat, brand_data in categorized.items():
        print(f"   - {cat}: {len(brand_data)} marcas")

    # --- Exportar a CSV ---
    csv_file = output_dir / "brands_summary.csv"
    with open(csv_file, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["marca", "categoría", "cantidad_colores"])
        for cat, brand_data in categorized.items():
            for brand, colors in brand_data.items():
                writer.writerow([brand, cat, len(colors)])

    print(f"\n✅ JSONs y resumen CSV generados en la carpeta: {output_dir}\n")


if __name__ == "__main__":
    main()



📊 Resumen de categorización:
   Total de marcas detectadas: 649
   - Automotive: 7 marcas
   - Fashion: 6 marcas
   - Finance: 8 marcas
   - Food: 7 marcas
   - Healthcare: 4 marcas
   - Media: 6 marcas
   - Retail: 8 marcas
   - Tech: 22 marcas
   - Telecom: 3 marcas
   - Travel: 7 marcas
   - Uncategorized: 571 marcas

✅ JSONs y resumen CSV generados en la carpeta: output_json



## El tema de lista de BRAND_COLORS se me mezcló después de tantas vueltas
- voy a tratar de webscrapping la página
- Entiendo que podría hacer la descarga con Web Scrapping pero por ahora lo vamos a hacer con es descargar el archivo en un formato de texto (ofrece CSS, SCC,... ) y tocarlo con código para que quede el DIC que me interesa a mí

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import time # Para añadir un pequeño retraso entre solicitudes

# URL de la página principal que contiene los enlaces a las marcas
base_url = "https://brandcolors.net/"

# Diccionario para almacenar los colores
brand_colors = {}

try:
    # Descargar la página principal
    print("Descargando la página principal...")
    response = requests.get(base_url)
    response.raise_for_status()  # Lanza una excepción si la solicitud falla
    soup = BeautifulSoup(response.text, 'html.parser')

    # Encontrar todos los enlaces a las páginas de marcas individuales.
    # Inspeccionando la página, los enlaces a las marcas individuales parecen estar
    # dentro de elementos <a> con el atributo 'href' que comienza con '/' y no es '/about'.
    # También suelen tener un <span> hijo con la clase 'color-box'.
    brand_links = soup.find_all('a', href=re.compile(r'^/(?!about)'))

    print(f"Encontrados {len(brand_links)} posibles enlaces a marcas.")

    # Iterar sobre cada enlace de marca encontrado
    for i, link in enumerate(brand_links):
        brand_slug = link.get('href').strip('/')
        brand_name_element = link.find('span', class_='brand-title')
        if brand_name_element:
            brand_name = brand_name_element.text.strip()
        else:
            # Si no hay span.brand-title, usar el slug como nombre temporal
            brand_name = brand_slug.replace('-', ' ').title()

        brand_url = f"{base_url}{brand_slug}"
        print(f"({i+1}/{len(brand_links)}) Procesando marca: {brand_name} ({brand_url})")

        try:
            # Añadir un pequeño retraso para ser cortés con el servidor
            time.sleep(0.2)
            
            # Descargar la página específica de la marca
            brand_response = requests.get(brand_url)
            brand_response.raise_for_status()
            brand_soup = BeautifulSoup(brand_response.text, 'html.parser')

            # Encontrar todos los bloques de color para esta marca.
            # Estos están dentro de elementos <div> con la clase 'color-item'.
            color_items = brand_soup.find_all('div', class_='color-item')

            if not color_items:
                print(f"  Advertencia: No se encontraron colores para {brand_name}")
                continue

            # Iterar sobre cada item de color
            for j, item in enumerate(color_items):
                # Extraer el código hexadecimal del color.
                # Está dentro de un <span> con la clase 'color-box'.
                color_box = item.find('span', class_='color-box')
                if color_box and color_box.get('style'):
                    # El estilo contiene algo como "background-color:#FF0000;"
                    style = color_box.get('style')
                    hex_code_match = re.search(r'background-color:\s*(#[0-9a-fA-F]{6})\s*;?', style)
                    if hex_code_match:
                        hex_code = hex_code_match.group(1)
                    else:
                        print(f"  Advertencia: No se pudo extraer el código hexadecimal del estilo: {style}")
                        continue
                else:
                    print(f"  Advertencia: No se encontró el elemento 'color-box' o su atributo 'style' para {brand_name}")
                    continue

                # Determinar la clave del diccionario
                if j == 0:
                    key = brand_name
                else:
                    key = f"{brand_name}{j+1}"

                # Agregar al diccionario
                brand_colors[key] = hex_code
                print(f"    Color encontrado: {key} -> {hex_code}")

        except requests.exceptions.RequestException as e:
            print(f"  Error al descargar la página de la marca {brand_name}: {e}")
        except Exception as e:
             print(f"  Error inesperado al procesar la marca {brand_name}: {e}")


    # Guardar el diccionario en un archivo Python
    if brand_colors:
        output_filename = "brand_colors.py"
        with open(output_filename, 'w', encoding='utf-8') as f: # Especificar encoding
            f.write("brand_colors = {\n")
            # Ordenar las claves para una salida más limpia
            for name in sorted(brand_colors.keys()): 
                code = brand_colors[name]
                # Escapar comillas simples y dobles en el nombre si es necesario
                safe_name = name.replace("\\", "\\\\").replace("'", "\\'").replace("\"", "\\\"")
                f.write(f"    '{safe_name}': '{code}',\n")
            f.write("}\n")
        
        print(f"\nDiccionario generado con éxito. Se guardó en '{output_filename}'")
        print(f"Número total de colores extraídos: {len(brand_colors)}")
        
        # Opcional: Imprimir una muestra del diccionario
        print("\nMuestra del diccionario generado:")
        sample_items = list(brand_colors.items())[:10] # Primeros 10 items
        for name, code in sample_items:
            print(f"  '{name}': '{code}',")
        if len(brand_colors) > 10:
             print("  ... (y más)")
             
    else:
        print("\nNo se extrajeron colores.")

except requests.exceptions.RequestException as e:
    print(f"Error al descargar la página principal: {e}")
except Exception as e:
    print(f"Error inesperado: {e}")


In [ ]:
BRAND_COLORS = {
 '365 DataScience': '#108A99',
 '4ormat': '#fb0a2a',
 '500px': '#02adea',
 'AMD': '#ED1C24', 'AMD-2': '#000000',
 'AOL': '#00c4ff',
 'ARM': '#0084AB',
 'AT&T': '#2D96C8',
 'About-me': '#00405d', 'About-me-2': '#062f3c', 'About-me-3': '#2b82ad', 'About-me-4': '#cc7a00', 'About-me-5': '#ffcc33',
 'Acer': '#83b81a',
 'Addvocate': '#ff3322',
 'Adobe': '#ff0000', 'Adobe-1': '#FF0000', 'Adobe-2': '#fbb034', 'Adobe-3': '#ffdd00', 'Adobe-4': '#c1d82f', 'Adobe-5': '#00a4e4', 'Adobe-6': '#8a7967', 'Adobe-7': '#6a737b',
 'Aim': '#ffd900',
 'Airbnb': '#FF5A60',
 'Alibaba': '#FF7300',
 'Amazon': '#ff9900', 'Amazon-1': '#FF9900', 'Amazon-2': '#146eb4',
 'American Express-1': '#006FCF', 'American Express-2': '#000000',
 'Android': '#a4c639',
 'Angies-list': '#7fbb00',
 'Aol': '#ff0b00', 'Aol-2': '#00c4ff',
 'Apple': '#777777', 'Apple-1': '#007AFF', 'Apple-2': '#000000', 'Apple-3': '#FFFFFF',
 'Arch-linux': '#1793d1', 'Arch-linux-2': '#333333',
 'Asana': '#1d8dd5', 'Asana-2': '#34ad00',
 'Atlassian': '#003366',
 'Audi-1': '#BB0A30', 'Audi-2': '#000000',
 'BBC': '#333333',
 'BMW-1': '#1C69D4', 'BMW-2': '#000000', 'BMW-3': '#FFFFFF',
 'BT': '#084897',
 'Barnes & Noble': '#295A33',
 'Basecamp': '#6bbd6d',
 'Beats': '#FF0000',
 'Behance': '#005cff',
 'Big-cartel': '#a0ac48', 'Big-cartel-2': '#70b29c',
 'Bing': '#ffb900',
 'Bitbucket': '#205081',
 'Bitly': '#ee6123', 'Bitly-2': '#61b3de',
 'Blizzard': '#01B2F1',
 'Blogger': '#fc4f08',
 'Boeing': '#0039a6',
 'Booking-com': '#003580',
 'Box': '#00aeef',
 'Broadcom': '#E81231',
 'Burger King-1': '#EC1C24', 'Burger King-2': '#FDBD10', 'Burger King-3': '#0066B2',
 'BuzzFeed': '#EE3322',
 'Cadbury': '#472F92',
 'Canon-1': '#BF0000', 'Canon-2': '#000000',
 'Carbonmade': '#613854',
 'Cheddar': '#ff7243',
 'China Mobile': '#1d64b8',
 'Cisco': '#11495E',
 'Coca-Cola-1': '#ED1C16', 'Coca-Cola-2': '#000000', 'Coca-Cola-3': '#FFFFFF', 'Coca-cola': '#ed1c16',
 'Code-school': '#616f67', 'Code-school-2': '#c68143',
 'Corning': '#00559B',
 'Creative-market': '#8ba753',
 'DELL': '#0085C3',
 'DHL-1': '#FFCD00', 'DHL-2': '#D40511',
 'Delicious': '#3399ff', 'Delicious-2': '#222222', 'Delicious-3': '#eeeeee',
 'Dell': '#0085c3', 'Dell-2': '#7ab800', 'Dell-3': '#f2af00', 'Dell-4': '#dc5034', 'Dell-5': '#ce1126', 'Dell-6': '#b7295a', 'Dell-7': '#6e2585', 'Dell-8': '#71c6c1', 'Dell-9': '#5482ab', 'Dell-10': '#009bbb', 'Dell-11': '#444444', 'Dell-12': '#eeeeee',
 'Designer News': '#2d72da', 'Designer-news': '#2d72d9',
 'Designmoo': '#e64b50', 'Designmoo-2': '#dbc65d',
 'Deviantart': '#4e6252',
 'Devour': '#ff0000',
 'Dewalt': '#febd17',
 'DirecTV': '#0097CD',
 'Discord-1': '#5865F2', 'Discord-2': '#57F287', 'Discord-3': '#FEE75C', 'Discord-4': '#ED4245',
 'Dish Network': '#DA121D',
 'Disqus': '#2e9fff',
 'Django': '#092e20',
 'Docker-1': '#0DB7ED', 'Docker-2': '#384D54',
 'Dribbble': '#ea4c89', 'Dribbble-2': '#ea4c89', 'Dribbble-3': '#8aba56', 'Dribbble-4': '#ff8833', 'Dribbble-5': '#00b6e3', 'Dribbble-6': '#9ba5a8',
 'Dropbox': '#007ee5', 'Dropbox-2': '#7b8994', 'Dropbox-3': '#47525d', 'Dropbox-4': '#3d464d',
 'Drupal': '#0077c0', 'Drupal-2': '#81ceff', 'Drupal-3': '#00598e',
 'Dunked': '#2f3238', 'Dunked-2': '#3fc380',
 'Dyson': '#010101',
 'E4': '#6D1D7C',
 'ESPN': '#FF0033',
 'Ebay': '#e53238', 'Ebay-2': '#0064d2', 'Ebay-3': '#f5af02', 'Ebay-4': '#86b817',
 'Eclass': '#cc0000',
 'Ember': '#f05e1b',
 'Engadget': '#00bdf6',
 'Envato': '#528036',
 'Ericsson': '#002561',
 'Etsy': '#eb6d20',
 'Evernote': '#7ac142',
 'Expedia': '#003050',
 'FIFA-1': '#326295', 'FIFA-2': '#000000',
 'Fab-com': '#dd0017',
 'Facebook': '#3b5998',
 'Fairmont-1': '#A6A685', 'Fairmont-2': '#E32119',
 'Fayerwayer': '#ff3300',
 'FedEx': '#4D148C', 'FedEx-1': '#4D148C', 'FedEx-2': '#FF6600',
 'Ferrari': '#1C396D',
 'Firefox': '#e66000',
 'Fitbit': '#45C2C5',
 'Flattr': '#f67c1a', 'Flattr-2': '#338d11',
 'Flickr': '#0063dc', 'Flickr-2': '#ff0084',
 'Flipboard': '#C10000',
 'Flipkart': '#005387',
 'Ford': '#4078C0',
 'Forrst': '#5b9a68',
 'Foursquare': '#0cbadf',
 'Foxconn': '#1E5AA0',
 'Garmin': '#007cc3',
 'General Electric': '#019DDD',
 'Geocaching': '#4a742c',
 'Getglue': '#2d75a2',
 'Gimmebar': '#f70078',
 'GitHub': '#171515', 'GitHub-1': '#FCA326',
 'GitLab-1': '#006699',
 'Github': '#171515',
 'Gittip': '#663300', 'Gittip-2': '#339966',
 'GoPro': '#009EE2',
 'Google': '#4285f4', 'Google-1': '#4285F4', 'Google-2': '#db4437', 'Google-3': '#f4b400', 'Google-4': '#0f9d58', 'Google-5': '#e7e6dd',
 'Googleplus': '#dd4b39',
 'Grooveshark': '#f77f00',
 'Groupon': '#82b548',
 'HP': '#0096d6', 'HP-1': '#0096D6', 'HP-2': '#D7410B',
 'HTC': '#69B40F',
 'Hacker News': '#ff6600', 'Hacker-news': '#ff6600',
 'Hellowallet': '#0085ca',
 'Heroku': '#c7c5e6', 'Heroku-2': '#6567a5',
 'Home Depot': '#F86201',
 'Honda-1': '#CC0000', 'Honda-2': '#000000',
 'Hootsuite': '#003366',
 'Houzz': '#73ba37',
 'Hp': '#0096d6',
 'Hsbc': '#db0011',
 'Html5': '#ec6231',
 'Huawei': '#D62D24',
 'Hulu': '#8cc83b',
 'IBM': '#003e6a',
 'IKEA': '#ffcc33', 'IKEA-1': '#FF6600',
 'IMDb': '#f3ce13',
 'Ibm': '#003e6a',
 'Ikea': '#ffcc33',
 'Imdb': '#f3ce13',
 'Imgur': '#85bf25',
 'Instagram': '#3f729b', 'Instagram-1': '#E4405F', 'Instagram-2': '#405DE6', 'Instagram-3': '#5851DB', 'Instagram-4': '#833AB4', 'Instagram-5': '#C13584', 'Instagram-6': '#E1306C', 'Instagram-7': '#FD1D1D', 'Instagram-8': '#F56040', 'Instagram-9': '#FCAF45',
 'Instapaper': '#1c1c1c',
 'Intel': '#0071c5', 'Intel-1': '#0071C5', 'Intel-2': '#00AEEF',
 'Intuit': '#365ebf',
 'JBL': '#9CB443',
 'Jawbone': '#1A1A1A',
 'KFC-1': '#F40027', 'KFC-2': '#000000',
 'Khan Academy-1': '#242F3A', 'Khan Academy-2': '#DDB321',
 'Kickstarter': '#76cc1e',
 'Kippt': '#e03500',
 'Kiwipay': '#00b0df',
 'Kodery': '#00af81',
 'LG': '#B20E50', 'LG-1': '#A50034', 'LG-2': '#C4A484',
 'LINE': '#1DCD00',
 'Lamborghini': '#0A66C2',
 'Lastfm': '#c3000d',
 'LinkedIn': '#0e76a8', 'LinkedIn-1': '#0A3A6C',
 'Linkedin': '#007bb6',
 'Livestream': '#cf0005',
 'Lukkin': '#c90000',
 'Lumo': '#576396',
 'Lyft': '#00B4AE', 'Lyft-1': '#FF00BF', 'Lyft-2': '#352384',
 'MTM-1': '#113D76', 'MTM-2': '#D6682D', 'MTM-3': '#D8630E', 'MTM-4': '#00A1F1',
 'Makita': '#d82028', 'Makita-2': '#29a0b7',
 'Mastercard-1': '#FF5F00', 'Mastercard-2': '#EB001B', 'Mastercard-3': '#F79E1B',
 "McDonald's-1": '#FFC72C', "McDonald's-2": '#DA020E',
 'Meetup': '#e51937',
 'Mercedes-Benz-1': '#00ADEF', 'Mercedes-Benz-2': '#000000',
 'Mic': '#21c1ff',
 'Microsoft': '#00a1f1', 'Microsoft-2': '#7cbb00', 'Microsoft-3': '#00a1f1', 'Microsoft-4': '#ffbb00',
 'Microsoft-office': '#ea3e23',
 'Mixpanel': '#00a9d2',
 'MongoDB': '#5C92FA',
 'Motorola': '#5C92FA',
 'Mozilla': '#C34139',
 'MySQL-1': '#F29111', 'MySQL-2': '#FFCC00',
 'NBA-1': '#1D428A', 'NBA-2': '#C8102E', 'NBA-3': '#000000',
 'NFL-1': '#013369', 'NFL-2': '#D50A0A',
 'NTT DoCoMo': '#CC0033',
 'NVIDIA': '#77B900', 'NVIDIA-1': '#76B900', 'NVIDIA-2': '#000000',
 'National Geographic-1': '#000000', 'National Geographic-2': '#08107B',
 'Nest': '#1EB6DC',
 'Netflix': '#b9070a', 'Netflix-1': '#E50914', 'Netflix-2': '#221F1F',
 'Nikon-1': '#FFD700', 'Nikon-2': '#000000',
 'Nintendo': '#8C8C8C', 'Nintendo-1': '#E60012', 'Nintendo-2': '#8C8C8C',
 'Nokia': '#183693',
 'Nvidia': '#76b900',
 'Odnoklassniki': '#ed812b',
 'Olympics-1': '#0081C8', 'Olympics-2': '#FCB131', 'Olympics-3': '#EE334E', 'Olympics-4': '#00A651', 'Olympics-5': '#000000',
 'Olympus-1': '#DFB226', 'Olympus-2': '#777777', 'Olympus-3': '#8892BE',
 'Opera': '#cc0f16',
 'Oracle-1': '#F80000', 'Oracle-2': '#7F7F7F',
 'PHP-1': '#4F5B93', 'PHP-2': '#99CC99', 'PHP-3': '#FFDE57',
 'Panasonic': '#0438C2',
 'Path': '#e41f11',
 'PayPal': '#3b7bbf', 'PayPal-1': '#0070BA', 'PayPal-2': '#003087', 'PayPal-3': '#009CDE', 'PayPal-4': '#012169',
 'Paypal-dark': '#1e477a', 'Paypal-dark-2': '#3b7bbf',
 'Pepsi-1': '#004B93', 'Pepsi-2': '#E32934', 'Pepsi-3': '#FFFFFF',
 'Philips': '#0A5DD7',
 'Pinboard': '#0000e6',
 'Pinterest': '#cc2127',
 'Playstation': '#665cbe',
 'Pocket': '#ee4056',
 'Prezi': '#318bff',
 'Priceline': '#0A84C1',
 'Product Hunt': '#DA552F',
 'Pusha': '#0f71b4',
 'Python-1': '#4584B6', 'Python-2': '#646464', 'Python-3': '#25D366',
 'Qualcomm': '#005daa',
 'Quora': '#a82400',
 'Quote-fm': '#66ceff',
 'Rdio': '#007dc3',
 'Readability': '#9c0000',
 'Red-hat': '#cc0000',
 'Reddit': '#cee3f8', 'Reddit-2': '#ff4500',
 'Redfin': '#A02021',
 'Resource': '#7eb400',
 'Rockpack': '#0ba6ab',
 'Roku': '#662E93',
 'Roon': '#62b0d9',
 'Rovio': '#C02227',
 'Rss': '#ee802f',
 'Salesforce': '#1798c1', 'Salesforce-1': '#00A1E0', 'Salesforce-2': '#16325C', 'Salesforce-3': '#3EBAFF',
 'Samsung': '#0c4da2', 'Samsung-1': '#1428A0', 'Samsung-2': '#000000',
 'Shazam': '#1B87E3',
 'Shopify': '#96bf48', 'Shopify-1': '#96BF48', 'Shopify-2': '#5E8E3E', 'Shopify-3': '#7AB55C',
 'Skype': '#00aff0',
 'Slack': '#6fcbdc', 'Slack-1': '#4A154B', 'Slack-2': '#e9a821', 'Slack-3': '#e11664', 'Slack-4': '#3fba91', 'Slack-5': '#E01E5A',
 'Smashing-magazine': '#f0503a',
 'Snagajob': '#f47a20',
 'Snapchat': '#FFFC00',
 'SoftBank': '#fbbd09',
 'Softonic': '#008ace',
 'Sony-1': '#000000', 'Sony-2': '#0070D1',
 'SoundCloud': '#FF8800',
 'Soundcloud': '#ff7700',
 'Space-box': '#f86960',
 'Spotify': '#81b71a', 'Spotify-1': '#1DB954', 'Spotify-2': '#191414',
 'Sprint': '#fee100',
 'Squarespace': '#121212',
 'Stack Overflow': '#ef8236', 'StackOverflow': '#ef8236', 'Stackoverflow': '#ef8236',
 'Staples': '#cc0000',
 'Starbucks': '#00704A', 'Starbucks-1': '#00704A', 'Starbucks-2': '#000000',
 'Status-chart': '#d7584f',
 'Strava': '#fc4c02',
 'Stripe': '#008cdd', 'Stripe-1': '#635BFF', 'Stripe-2': '#0A2540',
 'Studyblue': '#00afe1',
 'Stumbleupon': '#f74425',
 'Subway-1': '#00543C', 'Subway-2': '#FFC72C',
 'Swarm': '#fd9627',
 'T-Mobile': '#ea0a8e', 'T-mobile': '#ea0a8e', 'TMobile': '#ea0a8e',
 'Technorati': '#40a800',
 'Tesla': '#CC0000', 'Tesla-1': '#CC0000', 'Tesla-2': '#000000',
 'The-next-web': '#ef4423',
 'TikTok-1': '#FF0050', 'TikTok-2': '#00F2EA', 'TikTok-3': '#000000',
 'Toyota-1': '#EB0A1E', 'Toyota-2': '#000000',
 'Treehouse': '#5cb868',
 'Trello': '#256a92',
 'Trulia': '#5eab1f',
 'Tumblr': '#34526f',
 'Twitch': '#6441a5', 'Twitch-1': '#9146FF', 'Twitch-2': '#772CE8', 'Twitch-tv': '#6441a5',
 'Twitter': '#55acee', 'Twitter-1': '#1DA1F2', 'Twitter-2': '#14171A',
 'Typekit': '#9aca3c',
 'Typo3': '#ff8700',
 'UPS-1': '#8A6914', 'UPS-2': '#461A00',
 'Uber': '#1CA8C3', 'Uber-1': '#000000', 'Uber-2': '#FFFFFF',
 'Ubuntu': '#dd4814',
 'Ustream': '#3388ff',
 'Utorrent': '#00853f',
 'VSCO': '#AAA94C',
 'Verizon': '#ef1d1d',
 'Vimeo': '#44bbff',
 'Vine': '#00a478',
 'Virb': '#06afd8',
 'Virgin-media': '#cc0000',
 'Visa': '#0157a2', 'Visa-1': '#1A1F71', 'Visa-2': '#F79100',
 'Vkontakte': '#45668e',
 'Vodafone': '#E90000',
 'Volkswagen-1': '#00B1EB', 'Volkswagen-2': '#001E50',
 'Walmart': '#005CB0',
 'WeChat': '#93D034',
 'Western Digital-5': '#003369', 'Western Digital-6': '#FF0000',
 'WhatsApp': '#34AF23',
 'Windows': '#00bcf2',
 'Windows-phone': '#68217a',
 'Wolfram-alpha': '#f68d1e',
 'Wooga': '#5b009c',
 'WordPress': '#464646', 'Wordpress': '#21759b', 'Wordpress-2': '#d54e21', 'Wordpress-3': '#464646', 'Wordpress-com': '#1e8cbe',
 'Wunderlist': '#2b88d9',
 'XBOX': '#9bc848',
 'Xbox': '#52b043',
 'Xiaomi': '#FF4A03',
 'Xing': '#126567',
 'Yahoo': '#720e9e',
 'Yandex': '#ffcc00',
 'Yelp': '#c41200',
 'Yo': '#9B59B6',
 'YouTube': '#e52d27', 'YouTube-1': '#FF0000', 'YouTube-2': '#282828', 'Youtube': '#cd201f',
 'ZTE': '#0A50A0',
 'Zalongo': '#5498dc',
 'Zendesk': '#78a300',
 'Zerply': '#9dcc7a',
 'Zillow': '#0079E4',
 'Zoom-1': '#2D8CFF', 'Zoom-2': '#9B9B9B',
 'Zootool': '#5e8b1d',
 'Zopim': '#ff9d3b',
 'about.me': '#00405d',
 'bitly': '#ee6123',
 'eBay': '#e53238',
 'iHeartRadio': '#BC1C10'
}

In [ ]:
## Ordenar diccionario alfabéticamente sin distinción de MAY/MIN...
sorted_keys = sorted(BRAND_COLORS.keys(), key = lambda s: s.lower())
sorted_bc = {k: BRAND_COLORS[k] for k in sorted_keys}
sorted_bc

In [ ]:
BRAND_COLORS = {
    '365 DataScience': '#108A99',
 '4ormat': '#fb0a2a',
 '500px': '#02adea',
 'About-me': '#00405d',
 'About-me-2': '#062f3c',
 'About-me-3': '#2b82ad',
 'About-me-4': '#cc7a00',
 'About-me-5': '#ffcc33',
 'Acer': '#83b81a',
 'Addvocate': '#ff3322',
 'Adobe': '#ff0000',
 'Adobe-2': '#fbb034',
 'Adobe-3': '#ffdd00',
 'Adobe-4': '#c1d82f',
 'Adobe-5': '#00a4e4',
 'Adobe-6': '#8a7967',
 'Adobe-7': '#6a737b',
 'Aim': '#ffd900',
 'Airbnb': '#FF5A60',
 'Alibaba': '#FF7300',
 'Amazon': '#ff9900',
 'Amazon-2': '#146eb4',
 'AMD': '#ED1C24',
 'AMD-2': '#000000',
 'American Express': '#006FCF',
 'American Express-2': '#000000',
 'Android': '#a4c639',
 'Angies-list': '#7fbb00',
 'AOL': '#00c4ff',
 'AOL-2': '#ff0b00',
 'AOL-3': '#00c4ff',
 'Apple': '#777777',
 'Apple-2': '#007AFF',
 'Apple-3': '#000000',
 'Apple-4': '#FFFFFF',
 'Arch-linux': '#1793d1',
 'Arch-linux-2': '#333333',
 'ARM': '#0084AB',
 'Asana': '#1d8dd5',
 'Asana-2': '#34ad00',
 'AT&T': '#2D96C8',
 'Atlassian': '#003366',
 'Audi': '#BB0A30',
 'Audi-2': '#000000',
 'Barnes & Noble': '#295A33',
 'Basecamp': '#6bbd6d',
 'BBC': '#333333',
 'Beats': '#FF0000',
 'Behance': '#005cff',
 'Big-cartel': '#a0ac48',
 'Big-cartel-2': '#70b29c',
 'Bing': '#ffb900',
 'Bitbucket': '#205081',
 'Bitly': '#ee6123',
 'Bitly-2': '#61b3de',
 'Blizzard': '#01B2F1',
 'Blogger': '#fc4f08',
 'BMW': '#1C69D4',
 'BMW-2': '#000000',
 'BMW-3': '#FFFFFF',
 'Boeing': '#0039a6',
 'Booking-com': '#003580',
 'Box': '#00aeef',
 'Broadcom': '#E81231',
 'BT': '#084897',
 'Burger King': '#EC1C24',
 'Burger King-2': '#FDBD10',
 'Burger King-3': '#0066B2',
 'BuzzFeed': '#EE3322',
 'Cadbury': '#472F92',
 'Canon': '#BF0000',
 'Canon-2': '#000000',
 'Carbonmade': '#613854',
 'Cheddar': '#ff7243',
 'China Mobile': '#1d64b8',
 'Cisco': '#11495E',
 'Coca-cola': '#ed1c16',
 'Coca-Cola-2': '#000000',
 'Coca-Cola-3': '#FFFFFF',
 'Code-school': '#616f67',
 'Code-school-2': '#c68143',
 'Corning': '#00559B',
 'Creative-market': '#8ba753',
 'Delicious': '#3399ff',
 'Delicious-2': '#222222',
 'Delicious-3': '#eeeeee',
 'DELL': '#0085C3',
 'DELL-2': '#7ab800',
 'DELL-3': '#f2af00',
 'DELL-4': '#dc5034',
 'DELL-5': '#ce1126',
 'DELL-6': '#b7295a',
 'DELL-7': '#6e2585',
 'DELL-8': '#71c6c1',
 'DELL-9': '#5482ab',
 'DELL-10': '#009bbb',
 'DELL-11': '#444444',
 'DELL-12': '#eeeeee',
 'Designer News': '#2d72d9',
 'Designmoo': '#e64b50',
 'Designmoo-2': '#dbc65d',
 'Deviantart': '#4e6252',
 'Devour': '#ff0000',
 'Dewalt': '#febd17',
 'DHL': '#FFCD00',
 'DHL-2': '#D40511',
 'DirecTV': '#0097CD',
 'Discord': '#5865F2',
 'Discord-2': '#57F287',
 'Discord-3': '#FEE75C',
 'Discord-4': '#ED4245',
 'Dish Network': '#DA121D',
 'Disqus': '#2e9fff',
 'Django': '#092e20',
 'Docker': '#0DB7ED',
 'Docker-2': '#384D54',
 'Dribbble': '#ea4c89',
 'Dribbble-2': '#ea4c89',
 'Dribbble-3': '#8aba56',
 'Dribbble-4': '#ff8833',
 'Dribbble-5': '#00b6e3',
 'Dribbble-6': '#9ba5a8',
 'Dropbox': '#007ee5',
 'Dropbox-2': '#7b8994',
 'Dropbox-3': '#47525d',
 'Dropbox-4': '#3d464d',
 'Drupal': '#0077c0',
 'Drupal-2': '#81ceff',
 'Drupal-3': '#00598e',
 'Dunked': '#2f3238',
 'Dunked-2': '#3fc380',
 'Dyson': '#010101',
 'E4': '#6D1D7C',
 'eBay': '#e53238',
 'ebay-2': '#0064d2',
 'ebay-3': '#f5af02',
 'ebay-4': '#86b817',
 'Eclass': '#cc0000',
 'Ember': '#f05e1b',
 'Engadget': '#00bdf6',
 'Envato': '#528036',
 'Ericsson': '#002561',
 'ESPN': '#FF0033',
 'Etsy': '#eb6d20',
 'Evernote': '#7ac142',
 'Expedia': '#003050',
 'Fab-com': '#dd0017',
 'Facebook': '#3b5998',
 'Fairmont': '#A6A685',
 'Fairmont-2': '#E32119',
 'Fayerwayer': '#ff3300',
 'FedEx': '#4D148C',
 'FedEx-2': '#FF6600',
 'Ferrari': '#1C396D',
 'FIFA-1': '#326295',
 'FIFA-2': '#000000',
 'Firefox': '#e66000',
 'Fitbit': '#45C2C5',
 'Flattr': '#f67c1a',
 'Flattr-2': '#338d11',
 'Flickr': '#0063dc',
 'Flickr-2': '#ff0084',
 'Flipboard': '#C10000',
 'Flipkart': '#005387',
 'Ford': '#4078C0',
 'Forrst': '#5b9a68',
 'Foursquare': '#0cbadf',
 'Foxconn': '#1E5AA0',
 'Garmin': '#007cc3',
 'General Electric': '#019DDD',
 'Geocaching': '#4a742c',
 'Getglue': '#2d75a2',
 'Gimmebar': '#f70078',
 'GitHub': '#171515',
 'Github': '#171515',
 'GitHub-1': '#FCA326',
 'GitLab-1': '#006699',
 'Gittip': '#663300',
 'Gittip-2': '#339966',
 'Google': '#4285f4',
 'Google-2': '#db4437',
 'Google-3': '#f4b400',
 'Google-4': '#0f9d58',
 'Google-5': '#e7e6dd',
 'Googleplus': '#dd4b39',
 'GoPro': '#009EE2',
 'Grooveshark': '#f77f00',
 'Groupon': '#82b548',
 'Hacker News': '#ff6600',
 'Hellowallet': '#0085ca',
 'Heroku': '#c7c5e6',
 'Heroku-2': '#6567a5',
 'Home Depot': '#F86201',
 'Honda': '#CC0000',
 'Honda-2': '#000000',
 'Hootsuite': '#003366',
 'Houzz': '#73ba37',
 'HP': '#0096d6',
 'HP-1': '#0096D6',
 'HP-2': '#D7410B',
 'Hsbc': '#db0011',
 'HTC': '#69B40F',
 'Html5': '#ec6231',
 'Huawei': '#D62D24',
 'Hulu': '#8cc83b',
 'IBM': '#003e6a',
 'Ibm': '#003e6a',
 'iHeartRadio': '#BC1C10',
 'IKEA': '#ffcc33',
 'Ikea': '#ffcc33',
 'IKEA-1': '#FF6600',
 'IMDb': '#f3ce13',
 'Imdb': '#f3ce13',
 'Imgur': '#85bf25',
 'Instagram': '#3f729b',
 'Instagram-2': '#E4405F',
 'Instagram-3': '#405DE6',
 'Instagram-4': '#5851DB',
 'Instagram-5': '#833AB4',
 'Instagram-6': '#C13584',
 'Instagram-7': '#E1306C',
 'Instagram-8': '#FD1D1D',
 'Instagram-9': '#F56040',
 'Instagram-10': '#FCAF45',
 'Instapaper': '#1c1c1c',
 'Intel': '#0071c5',
 'Intel-2': '#00AEEF',
 'Intuit': '#365ebf',
 'Jawbone': '#1A1A1A',
 'JBL': '#9CB443',
 'KFC': '#F40027',
 'KFC-2': '#000000',
 'Khan Academy': '#242F3A',
 'Khan Academy-2': '#DDB321',
 'Kickstarter': '#76cc1e',
 'Kippt': '#e03500',
 'Kiwipay': '#00b0df',
 'Kodery': '#00af81',
 'Lamborghini': '#0A66C2',
 'Lastfm': '#c3000d',
 'LG': '#B20E50',
 'LG-2': '#C4A484',
 'LINE': '#1DCD00',
 'LinkedIn': '#0e76a8',
 'Linkedin': '#007bb6',
 'LinkedIn-1': '#0A3A6C',
 'Livestream': '#cf0005',
 'Lukkin': '#c90000',
 'Lumo': '#576396',
 'Lyft': '#00B4AE',
 'Lyft-1': '#FF00BF',
 'Lyft-2': '#352384',
 'Makita': '#d82028',
 'Makita-2': '#29a0b7',
 'Mastercard': '#FF5F00',
 'Mastercard-2': '#EB001B',
 'Mastercard-3': '#F79E1B',
 "McDonald's-1": '#FFC72C',
 "McDonald's-2": '#DA020E',
 'Meetup': '#e51937',
 'Mercedes-Benz': '#00ADEF',
 'Mercedes-Benz-2': '#000000',
 'Mic': '#21c1ff',
 'Microsoft': '#00a1f1',
 'Microsoft-2': '#7cbb00',
 'Microsoft-3': '#00a1f1',
 'Microsoft-4': '#ffbb00',
 'Microsoft-office': '#ea3e23',
 'Mixpanel': '#00a9d2',
 'MongoDB': '#5C92FA',
 'Motorola': '#5C92FA',
 'Mozilla': '#C34139',
 'MTM': '#113D76',
 'MTM-2': '#D6682D',
 'MTM-3': '#D8630E',
 'MTM-4': '#00A1F1',
 'MySQL': '#F29111',
 'MySQL-2': '#FFCC00',
 'National Geographic': '#000000',
 'National Geographic-2': '#08107B',
 'NBA': '#1D428A',
 'NBA-2': '#C8102E',
 'NBA-3': '#000000',
 'Nest': '#1EB6DC',
 'Netflix': '#b9070a',
 'Netflix-1': '#E50914',
 'Netflix-2': '#221F1F',
 'NFL-1': '#013369',
 'NFL-2': '#D50A0A',
 'Nikon': '#FFD700',
 'Nikon-2': '#000000',
 'Nintendo': '#8C8C8C',
 'Nintendo-1': '#E60012',
 'Nintendo-2': '#8C8C8C',
 'Nokia': '#183693',
 'NTT DoCoMo': '#CC0033',
 'NVIDIA': '#77B900',
 'Nvidia': '#76b900',
 'NVIDIA-1': '#76B900',
 'NVIDIA-2': '#000000',
 'Odnoklassniki': '#ed812b',
 'Olympics-1': '#0081C8',
 'Olympics-2': '#FCB131',
 'Olympics-3': '#EE334E',
 'Olympics-4': '#00A651',
 'Olympics-5': '#000000',
 'Olympus-1': '#DFB226',
 'Olympus-2': '#777777',
 'Olympus-3': '#8892BE',
 'Opera': '#cc0f16',
 'Oracle-1': '#F80000',
 'Oracle-2': '#7F7F7F',
 'Panasonic': '#0438C2',
 'Path': '#e41f11',
 'PayPal': '#3b7bbf',
 'PayPal-1': '#0070BA',
 'PayPal-2': '#003087',
 'PayPal-3': '#009CDE',
 'PayPal-4': '#012169',
 'Paypal-dark': '#1e477a',
 'Paypal-dark-2': '#3b7bbf',
 'Pepsi-1': '#004B93',
 'Pepsi-2': '#E32934',
 'Pepsi-3': '#FFFFFF',
 'Philips': '#0A5DD7',
 'PHP-1': '#4F5B93',
 'PHP-2': '#99CC99',
 'PHP-3': '#FFDE57',
 'Pinboard': '#0000e6',
 'Pinterest': '#cc2127',
 'Playstation': '#665cbe',
 'Pocket': '#ee4056',
 'Prezi': '#318bff',
 'Priceline': '#0A84C1',
 'Product Hunt': '#DA552F',
 'Pusha': '#0f71b4',
 'Python-1': '#4584B6',
 'Python-2': '#646464',
 'Python-3': '#25D366',
 'Qualcomm': '#005daa',
 'Quora': '#a82400',
 'Quote-fm': '#66ceff',
 'Rdio': '#007dc3',
 'Readability': '#9c0000',
 'Red-hat': '#cc0000',
 'Reddit': '#cee3f8',
 'Reddit-2': '#ff4500',
 'Redfin': '#A02021',
 'Resource': '#7eb400',
 'Rockpack': '#0ba6ab',
 'Roku': '#662E93',
 'Roon': '#62b0d9',
 'Rovio': '#C02227',
 'Rss': '#ee802f',
 'Salesforce': '#1798c1',
 'Salesforce-1': '#00A1E0',
 'Salesforce-2': '#16325C',
 'Salesforce-3': '#3EBAFF',
 'Samsung': '#0c4da2',
 'Samsung-1': '#1428A0',
 'Samsung-2': '#000000',
 'Shazam': '#1B87E3',
 'Shopify': '#96bf48',
 'Shopify-1': '#96BF48',
 'Shopify-2': '#5E8E3E',
 'Shopify-3': '#7AB55C',
 'Skype': '#00aff0',
 'Slack': '#6fcbdc',
 'Slack-1': '#4A154B',
 'Slack-2': '#e9a821',
 'Slack-3': '#e11664',
 'Slack-4': '#3fba91',
 'Slack-5': '#E01E5A',
 'Smashing-magazine': '#f0503a',
 'Snagajob': '#f47a20',
 'Snapchat': '#FFFC00',
 'SoftBank': '#fbbd09',
 'Softonic': '#008ace',
 'Sony-1': '#000000',
 'Sony-2': '#0070D1',
 'SoundCloud': '#FF8800',
 'Soundcloud': '#ff7700',
 'Space-box': '#f86960',
 'Spotify': '#81b71a',
 'Spotify-1': '#1DB954',
 'Spotify-2': '#191414',
 'Sprint': '#fee100',
 'Squarespace': '#121212',
 'Stack Overflow': '#ef8236',
 'StackOverflow': '#ef8236',
 'Stackoverflow': '#ef8236',
 'Staples': '#cc0000',
 'Starbucks': '#00704A',
 'Starbucks-1': '#00704A',
 'Starbucks-2': '#000000',
 'Status-chart': '#d7584f',
 'Strava': '#fc4c02',
 'Stripe': '#008cdd',
 'Stripe-1': '#635BFF',
 'Stripe-2': '#0A2540',
 'Studyblue': '#00afe1',
 'Stumbleupon': '#f74425',
 'Subway-1': '#00543C',
 'Subway-2': '#FFC72C',
 'Swarm': '#fd9627',
 'T-Mobile': '#ea0a8e',
 'T-mobile': '#ea0a8e',
 'Technorati': '#40a800',
 'Tesla': '#CC0000',
 'Tesla-1': '#CC0000',
 'Tesla-2': '#000000',
 'The-next-web': '#ef4423',
 'TikTok-1': '#FF0050',
 'TikTok-2': '#00F2EA',
 'TikTok-3': '#000000',
 'TMobile': '#ea0a8e',
 'Toyota-1': '#EB0A1E',
 'Toyota-2': '#000000',
 'Treehouse': '#5cb868',
 'Trello': '#256a92',
 'Trulia': '#5eab1f',
 'Tumblr': '#34526f',
 'Twitch': '#6441a5',
 'Twitch-1': '#9146FF',
 'Twitch-2': '#772CE8',
 'Twitch-tv': '#6441a5',
 'Twitter': '#55acee',
 'Twitter-1': '#1DA1F2',
 'Twitter-2': '#14171A',
 'Typekit': '#9aca3c',
 'Typo3': '#ff8700',
 'Uber': '#1CA8C3',
 'Uber-1': '#000000',
 'Uber-2': '#FFFFFF',
 'Ubuntu': '#dd4814',
 'UPS-1': '#8A6914',
 'UPS-2': '#461A00',
 'Ustream': '#3388ff',
 'Utorrent': '#00853f',
 'Verizon': '#ef1d1d',
 'Vimeo': '#44bbff',
 'Vine': '#00a478',
 'Virb': '#06afd8',
 'Virgin-media': '#cc0000',
 'Visa': '#0157a2',
 'Visa-1': '#1A1F71',
 'Visa-2': '#F79100',
 'Vkontakte': '#45668e',
 'Vodafone': '#E90000',
 'Volkswagen-1': '#00B1EB',
 'Volkswagen-2': '#001E50',
 'VSCO': '#AAA94C',
 'Walmart': '#005CB0',
 'WeChat': '#93D034',
 'Western Digital-5': '#003369',
 'Western Digital-6': '#FF0000',
 'WhatsApp': '#34AF23',
 'Windows': '#00bcf2',
 'Windows-phone': '#68217a',
 'Wolfram-alpha': '#f68d1e',
 'Wooga': '#5b009c',
 'WordPress': '#464646',
 'Wordpress': '#21759b',
 'Wordpress-2': '#d54e21',
 'Wordpress-3': '#464646',
 'Wordpress-com': '#1e8cbe',
 'Wunderlist': '#2b88d9',
 'XBOX': '#9bc848',
 'Xbox': '#52b043',
 'Xiaomi': '#FF4A03',
 'Xing': '#126567',
 'Yahoo': '#720e9e',
 'Yandex': '#ffcc00',
 'Yelp': '#c41200',
 'Yo': '#9B59B6',
 'YouTube': '#e52d27',
 'Youtube': '#cd201f',
 'YouTube-1': '#FF0000',
 'YouTube-2': '#282828',
 'Zalongo': '#5498dc',
 'Zendesk': '#78a300',
 'Zerply': '#9dcc7a',
 'Zillow': '#0079E4',
 'Zoom-1': '#2D8CFF',
 'Zoom-2': '#9B9B9B',
 'Zootool': '#5e8b1d',
 'Zopim': '#ff9d3b',
 'ZTE': '#0A50A0'
}

In [ ]:
# De Perplexiti
import matplotlib as mpl
import matplotlib.colors as mcolors

# Your expanded brand colors dictionary
brand_perplexity = {
    '365 DataScience': '#108A99',
    'AMD(1)': '#ED1C24',
    'AMD(2)': '#000000',
    'AT&T': '#00A8E0',
    'Adobe(1)': '#FF0000',
    'Adobe(2)': '#FBB034',
    'Adobe(3)': '#FFDD00',
    'Adobe(4)': '#C1D82F',
    'Adobe(5)': '#00A4E4',
    'Airbnb': '#FD5C63',
    'Alibaba': '#FF6A00',
    'Amazon(1)': '#FF9900',
    'Amazon(2)': '#146EB4',
    'American Express(1)': '#006FCF',
    'American Express(2)': '#000000',
    'Android': '#A4C639',
    'Apple(1)': '#007AFF',
    'Apple(2)': '#000000',
    'Apple(3)': '#FFFFFF',
    'Audi(1)': '#BB0A30',
    'Audi(2)': '#000000',
    'BMW(1)': '#1C69D4',
    'BMW(2)': '#000000',
    'BMW(3)': '#FFFFFF',
    'Burger King(1)': '#EC1C24',
    'Burger King(2)': '#FDBD10',
    'Burger King(3)': '#0066B2',
    'Cadbury': '#472F92',
    'Canon(1)': '#BF0000',
    'Canon(2)': '#000000',
    'Coca-Cola(1)': '#ED1C16',
    'Coca-Cola(2)': '#000000',
    'Coca-Cola(3)': '#FFFFFF',
    'DELL': '#0085C3',
    'DHL(1)': '#FFCD00',
    'DHL(2)': '#D40511',
    'Discord(1)': '#5865F2',
    'Discord(2)': '#57F287',
    'Discord(3)': '#FEE75C',
    'Discord(4)': '#ED4245',
    'Django': '#092E20',
    'Docker(1)': '#0DB7ED',
    'Docker(2)': '#384D54',
    'E4': '#6D1D7C',
    'ESPN': '#FF0033',
    'FIFA(1)': '#326295',
    'FIFA(2)': '#000000',
    'Facebook': '#3B5998',
    'Fairmont(1)': '#A6A685',
    'Fairmont(2)': '#E32119',
    'FedEx(1)': '#4D148C',
    'FedEx(2)': '#FF6600',
    'Ferrari': '#1C396D',
    'Ford': '#4078C0',
    'GitHub(1)': '#FCA326',
    'GitLab(1)': '#006699',
    'Google(1)': '#4285F4',
    'Google(2)': '#34A853',
    'Google(3)': '#FBBC05',
    'Google(4)': '#EA4335',
    'HP(1)': '#0096D6',
    'HP(2)': '#D7410B',
    'Honda(1)': '#CC0000',
    'Honda(2)': '#000000',
    'IBM': '#F5CB39',
    'IKEA(1)': '#FF6600',
    'Instagram(1)': '#E4405F',
    'Instagram(2)': '#405DE6',
    'Instagram(3)': '#5851DB',
    'Instagram(4)': '#833AB4',
    'Instagram(5)': '#C13584',
    'Instagram(6)': '#E1306C',
    'Instagram(7)': '#FD1D1D',
    'Instagram(8)': '#F56040',
    'Instagram(9)': '#FCAF45',
    'Intel(1)': '#0071C5',
    'Intel(2)': '#00AEEF',
    'JBL': '#9CB443',
    'KFC(1)': '#F40027',
    'KFC(2)': '#000000',
    'Khan Academy(1)': '#242F3A',
    'Khan Academy(2)': '#DDB321',
    'LG(1)': '#A50034',
    'LG(2)': '#C4A484',
    'Lamborghini': '#0A66C2',
    'LinkedIn(1)': '#0A3A6C',
    'Lyft(1)': '#FF00BF',
    'Lyft(2)': '#352384',
    'MTM(1)': '#113D76',
    'MTM(2)': '#D6682D',
    'MTM(3)': '#D8630E',
    'MTM(4)': '#00A1F1',
    'Mastercard(1)': '#FF5F00',
    'Mastercard(2)': '#EB001B',
    'Mastercard(3)': '#F79E1B',
    'McDonald\'s(1)': '#FFC72C',
    'McDonald\'s(2)': '#DA020E',
    'Mercedes-Benz(1)': '#00ADEF',
    'Mercedes-Benz(2)': '#000000',
    'Microsoft': '#589636',
    'MongoDB': '#5C92FA',
    'Motorola': '#00758F',
    'MySQL(1)': '#F29111',
    'MySQL(2)': '#FFCC00',
    'NBA(1)': '#1D428A',
    'NBA(2)': '#C8102E',
    'NBA(3)': '#000000',
    'NFL(1)': '#013369',
    'NFL(2)': '#D50A0A',
    'NVIDIA(1)': '#76B900',
    'NVIDIA(2)': '#000000',
    'National Geographic(1)': '#000000',
    'National Geographic(2)': '#08107B',
    'Netflix(1)': '#E50914',
    'Netflix(2)': '#221F1F',
    'Nikon(1)': '#FFD700',
    'Nikon(2)': '#000000',
    'Nintendo(1)': '#E60012',
    'Nintendo(2)': '#8C8C8C',
    'Olympics(1)': '#0081C8',
    'Olympics(2)': '#FCB131',
    'Olympics(3)': '#EE334E',
    'Olympics(4)': '#00A651',
    'Olympics(5)': '#000000',
    'Olympus(1)': '#DFB226',
    'Olympus(2)': '#777777',
    'Olympus(3)': '#8892BE',
    'Oracle(1)': '#F80000',
    'Oracle(2)': '#7F7F7F',
    'PHP(1)': '#4F5B93',
    'PHP(2)': '#99CC99',
    'PHP(3)': '#FFDE57',
    'PayPal(1)': '#0070BA',
    'PayPal(2)': '#003087',
    'PayPal(3)': '#009CDE',
    'PayPal(4)': '#012169',
    'Pepsi(1)': '#004B93',
    'Pepsi(2)': '#E32934',
    'Pepsi(3)': '#FFFFFF',
    'Python(1)': '#4584B6',
    'Python(2)': '#646464',
    'Python(3)': '#25D366',
    'Salesforce(1)': '#00A1E0',
    'Salesforce(2)': '#16325C',
    'Salesforce(3)': '#3EBAFF',
    'Samsung(1)': '#1428A0',
    'Samsung(2)': '#000000',
    'Shopify(1)': '#96BF48',
    'Shopify(2)': '#5E8E3E',
    'Shopify(3)': '#7AB55C',
    'Slack(1)': '#4A154B',
    'Slack(2)': '#36C5F0',
    'Slack(3)': '#2EB67D',
    'Slack(4)': '#ECB22E',
    'Slack(5)': '#E01E5A',
    'Sony(1)': '#000000',
    'Sony(2)': '#0070D1',
    'Spotify(1)': '#1DB954',
    'Spotify(2)': '#191414',
    'Starbucks(1)': '#00704A',
    'Starbucks(2)': '#000000',
    'Stripe(1)': '#635BFF',
    'Stripe(2)': '#0A2540',
    'Subway(1)': '#00543C',
    'Subway(2)': '#FFC72C',
    'Tesla(1)': '#CC0000',
    'Tesla(2)': '#000000',
    'TikTok(1)': '#FF0050',
    'TikTok(2)': '#00F2EA',
    'TikTok(3)': '#000000',
    'Toyota(1)': '#EB0A1E',
    'Toyota(2)': '#000000',
    'Twitch(1)': '#9146FF',
    'Twitch(2)': '#772CE8',
    'Twitter(1)': '#1DA1F2',
    'Twitter(2)': '#14171A',
    'UPS(1)': '#8A6914',
    'UPS(2)': '#461A00',
    'Uber(1)': '#000000',
    'Uber(2)': '#FFFFFF',
    'Visa(1)': '#1A1F71',
    'Visa(2)': '#F79100',
    'Volkswagen(1)': '#00B1EB',
    'Volkswagen(2)': '#001E50',
    'Western Digital(5)': '#003369',
    'Western Digital(6)': '#FF0000',
    'WhatsApp': '#9D0A0E',
    'YouTube(1)': '#FF0000',
    'YouTube(2)': '#282828',
    'Zoom(1)': '#2D8CFF',
    'Zoom(2)': '#9B9B9B'
}

## https://github.com/safareli/brand-colors-json/blob/master/index.json
bc_json = {
  "4ormat": "#fb0a2a",
  "500px": "#00aeef",
  "about-me": "#00405d",
  "about-me-2": "#062f3c",
  "about-me-3": "#2b82ad",
  "about-me-4": "#cc7a00",
  "about-me-5": "#ffcc33",
  "addvocate": "#ff3322",
  "adobe": "#ff0000",
  "adobe-2": "#fbb034",
  "adobe-3": "#ffdd00",
  "adobe-4": "#c1d82f",
  "adobe-5": "#00a4e4",
  "adobe-6": "#8a7967",
  "adobe-7": "#6a737b",
  "aim": "#ffd900",
  "amazon": "#ff9900",
  "amazon-2": "#146eb4",
  "android": "#a4c639",
  "angies-list": "#7fbb00",
  "aol": "#ff0b00",
  "aol-2": "#00c4ff",
  "asana": "#1f8dd6",
  "asana-2": "#34ad00",
  "atlassian": "#003366",
  "basecamp": "#6bbd6d",
  "behance": "#1769ff",
  "big-cartel": "#a0ac48",
  "big-cartel-2": "#70b29c",
  "bitly": "#ee6123",
  "bitly-2": "#61b3de",
  "blogger": "#f57d00",
  "boeing": "#0039a6",
  "booking-com": "#003580",
  "box": "#1277bc",
  "carbonmade": "#613854",
  "cheddar": "#ff7243",
  "coca-cola": "#ed1c16",
  "code-school": "#616f67",
  "code-school-2": "#c68143",
  "creative-market": "#8ba753",
  "delicious": "#3399ff",
  "delicious-2": "#222222",
  "delicious-3": "#eeeeee",
  "dell": "#0085c3",
  "dell-2": "#7ab800",
  "dell-3": "#f2af00",
  "dell-4": "#dc5034",
  "dell-5": "#ce1126",
  "dell-6": "#b7295a",
  "dell-7": "#6e2585",
  "dell-8": "#71c6c1",
  "dell-9": "#5482ab",
  "dell-10": "#009bbb",
  "dell-11": "#444444",
  "dell-12": "#eeeeee",
  "designmoo": "#e64b50",
  "designmoo-2": "#dbc65d",
  "deviantart": "#4e6252",
  "designer-news": "#2d72d9",
  "devour": "#ff0000",
  "dewalt": "#febd17",
  "disqus": "#2e9fff",
  "dribbble": "#444444",
  "dribbble-2": "#ea4c89",
  "dribbble-3": "#8aba56",
  "dribbble-4": "#ff8833",
  "dribbble-5": "#00b6e3",
  "dribbble-6": "#9ba5a8",
  "dropbox": "#007ee5",
  "dropbox-2": "#7b8994",
  "dropbox-3": "#47525d",
  "dropbox-4": "#3d464d",
  "drupal": "#0077c0",
  "drupal-2": "#81ceff",
  "drupal-3": "#00598e",
  "dunked": "#2f3238",
  "dunked-2": "#3fc380",
  "eclass": "#cc0000",
  "ebay": "#e53238",
  "ebay-2": "#0064d2",
  "ebay-3": "#f5af02",
  "ebay-4": "#86b817",
  "engadget": "#3399cc",
  "envato": "#528036",
  "etsy": "#eb6d20",
  "evernote": "#5ba525",
  "fab-com": "#dd0017",
  "fayerwayer": "#ff3300",
  "facebook": "#3b5998",
  "firefox": "#e66000",
  "flickr": "#0063dc",
  "flickr-2": "#ff0084",
  "forrst": "#5b9a68",
  "foursquare": "#F94877",
  "garmin": "#007cc3",
  "getglue": "#2d75a2",
  "gimmebar": "#f70078",
  "github": "#171515",
  "google": "#4285f4",
  "google-2": "#db4437",
  "google-3": "#f4b400",
  "google-4": "#0f9d58",
  "google-5": "#e7e6dd",
  "googleplus": "#dd4b39",
  "grooveshark": "#f77f00",
  "groupon": "#82b548",
  "hacker-news": "#ff6600",
  "hellowallet": "#0085ca",
  "heroku": "#c7c5e6",
  "heroku-2": "#6567a5",
  "hootsuite": "#003366",
  "houzz": "#73ba37",
  "hp": "#0096d6",
  "html5": "#ec6231",
  "hulu": "#8cc83b",
  "ibm": "#003e6a",
  "ikea": "#ffcc33",
  "imdb": "#f3ce13",
  "imgur": "#85bf25",
  "instagram": "#3f729b",
  "instapaper": "#1c1c1c",
  "intel": "#0071c5",
  "intuit": "#365ebf",
  "kickstarter": "#76cc1e",
  "kippt": "#e03500",
  "kodery": "#00af81",
  "lastfm": "#c3000d",
  "linkedin": "#007bb6",
  "livestream": "#cf0005",
  "lumo": "#576396",
  "lukkin": "#c90000",
  "makita": "#d82028",
  "makita-2": "#29a0b7",
  "mixpanel": "#a086d3",
  "meetup": "#e51937",
  "netflix": "#b9070a",
  "nokia": "#183693",
  "nvidia": "#76b900",
  "odnoklassniki": "#ed812b",
  "opera": "#cc0f16",
  "path": "#e41f11",
  "paypal-dark": "#1e477a",
  "paypal-dark-2": "#3b7bbf",
  "pinboard": "#0000e6",
  "pinterest": "#cc2127",
  "playstation": "#665cbe",
  "pocket": "#ee4056",
  "prezi": "#318bff",
  "pusha": "#0f71b4",
  "quora": "#a82400",
  "quote-fm": "#66ceff",
  "rdio": "#008fd5",
  "readability": "#9c0000",
  "red-hat": "#cc0000",
  "reddit": "#cee2f8",
  "reddit-2": "#ff4500",
  "resource": "#7eb400",
  "rockpack": "#0ba6ab",
  "roon": "#62b0d9",
  "rss": "#ee802f",
  "salesforce": "#1798c1",
  "samsung": "#0c4da2",
  "shopify": "#96bf48",
  "skype": "#00aff0",
  "slack": "#6fcbdc",
  "slack-2": "#e9a821",
  "slack-3": "#e11664",
  "slack-4": "#3fba91",
  "smashing-magazine": "#f0503a",
  "snagajob": "#f47a20",
  "softonic": "#008ace",
  "soundcloud": "#ff7700",
  "space-box": "#f86960",
  "spotify": "#81b71a",
  "snapchat": "#fffc00",
  "sprint": "#fee100",
  "squarespace": "#121212",
  "stackoverflow": "#ef8236",
  "staples": "#cc0000",
  "status-chart": "#d7584f",
  "stripe": "#008cdd",
  "studyblue": "#00afe1",
  "stumbleupon": "#f74425",
  "swarm": "#fd9627",
  "t-mobile": "#ea0a8e",
  "technorati": "#40a800",
  "the-next-web": "#ef4423",
  "treehouse": "#5cb868",
  "trello": "#256a92",
  "trulia": "#5eab1f",
  "tumblr": "#35465c",
  "twitch-tv": "#6441a5",
  "twitter": "#55acee",
  "typekit": "#9aca3c",
  "typo3": "#ff8700",
  "ubuntu": "#dd4814",
  "ustream": "#3388ff",
  "verizon": "#ef1d1d",
  "vimeo": "#44bbff",
  "vine": "#00a478",
  "virb": "#06afd8",
  "virgin-media": "#cc0000",
  "vkontakte": "#45668e",
  "wolfram-alpha": "#f68d1e",
  "wooga": "#5b009c",
  "wordpress": "#21759b",
  "wordpress-2": "#d54e21",
  "wordpress-3": "#464646",
  "wunderlist": "#2b88d9",
  "xbox": "#52b043",
  "xing": "#126567",
  "yahoo": "#720e9e",
  "yandex": "#ffcc00",
  "yelp": "#c41200",
  "youtube": "#cd201f",
  "zalongo": "#5498dc",
  "zendesk": "#78a300",
  "zerply": "#9dcc7a",
  "zootool": "#5e8b1d",
  "microsoft": "#f65314",
  "microsoft-2": "#7cbb00",
  "microsoft-3": "#00a1f1",
  "microsoft-4": "#ffbb00",
  "windows": "#00bcf2",
  "microsoft-office": "#ea3e23",
  "windows-phone": "#68217a",
  "bing": "#ffb900",
  "kiwipay": "#00b0df",
  "wordpress-com": "#1e8cbe",
  "django": "#092e20",
  "geocaching": "#4a742c",
  "utorrent": "#00853f",
  "arch-linux": "#1793d1",
  "arch-linux-2": "#333333",
  "gittip": "#663300",
  "gittip-2": "#339966",
  "flattr": "#f67c1a",
  "flattr-2": "#338d11",
  "hsbc": "#db0011",
  "zopim": "#ff9d3b",
  "bitbucket": "#205081",
  "strava": "#fc4c02",
  "mic": "#21c1ff"
}

UIcolor = {
    "500px": "#02adea",
    "about.me": "#00405d",
    "Acer": "#83b81a",
    "Adobe": "#ff0000",
    "Airbnb": "#FF5A60",
    "Alibaba": "#FF7300",
    "Amazon": "#ff9900",
    "Android": "#a4c639",
    "Apple": "#777777",
    "AOL": "#00c4ff",
    "ARM": "#0084AB",
    "Asana": "#1d8dd5",
    "AT&T": "#2D96C8",
    "Atlassian": "#003366",
    "Barnes & Noble": "#295A33",
    "BBC": "#333333",
    "Beats": "#FF0000",
    "Behance": "#005cff",
    "Bing": "#ffb900",
    "bitly": "#ee6123",
    "Blizzard": "#01B2F1",
    "Blogger": "#fc4f08",
    "Box": "#00aeef",
    "Broadcom": "#E81231",
    "BT": "#084897",
    "BuzzFeed": "#EE3322",
    "China Mobile": "#1d64b8",
    "Cisco": "#11495E",
    "Corning": "#00559B",
    "Dell": "#0085c3",
    "Designer News": "#2d72da",
    "DirecTV": "#0097CD",
    "Dish Network": "#DA121D",
    "Dribbble": "#ea4c89",
    "Dropbox": "#007ee5",
    "Dyson": "#010101",
    "eBay": "#e53238",
    "Ember": "#f05e1b",
    "Engadget": "#00bdf6",
    "Ericsson": "#002561",
    "Etsy": "#eb6d20",
    "Evernote": "#7ac142",
    "Expedia": "#003050",
    "Facebook": "#3b5998",
    "FedEx": "#4D148C",
    "Firefox": "#e66000",
    "Fitbit": "#45C2C5",
    "Flickr": "#0063dc",
    "Flipboard": "#C10000",
    "Flipkart": "#005387",
    "Foxconn": "#1E5AA0",
    "Foursquare": "#0cbadf",
    "General Electric": "#019DDD",
    "GitHub": "#171515",
    "Google": "#4285f4",
    "GoPro": "#009EE2",
    "Hacker News": "#ff6600",
    "HP": "#0096d6",
    "Home Depot": "#F86201",
    "HTC": "#69B40F",
    "Huawei": "#D62D24",
    "Hulu": "#8cc83b",
    "IBM": "#003e6a",
    "iHeartRadio": "#BC1C10",
    "IKEA": "#ffcc33",
    "IMDb": "#f3ce13",
    "Instagram": "#3f729b",
    "Intel": "#0071c5",
    "Intuit": "#365ebf",
    "Jawbone": "#1A1A1A",
    "Kickstarter": "#76cc1e",
    "LG": "#B20E50",
    "LinkedIn": "#0e76a8",
    "LINE": "#1DCD00",
    "Lyft": "#00B4AE",
    "Microsoft": "#00a1f1",
    "Mixpanel": "#00a9d2",
    "Motorola": "#5C92FA",
    "Mozilla": "#C34139",
    "Nest": "#1EB6DC",
    "Netflix": "#b9070a",
    "Nintendo": "#8C8C8C",
    "Nokia": "#183693",
    "NTT DoCoMo": "#CC0033",
    "NVIDIA": "#77B900",
    "Opera": "#cc0f16",
    "Panasonic": "#0438C2",
    "PayPal": "#3b7bbf",
    "Pinterest": "#cc2127",
    "Philips": "#0A5DD7",
    "Priceline": "#0A84C1",
    "Product Hunt": "#DA552F",
    "Qualcomm": "#005daa",
    "Reddit": "#cee3f8",
    "Redfin": "#A02021",
    "Roku": "#662E93",
    "Rovio": "#C02227",
    "Rdio": "#007dc3",
    "Salesforce": "#1798c1",
    "Samsung": "#0c4da2",
    "Snapchat": "#FFFC00",
    "Shazam": "#1B87E3",
    "Skype": "#00aff0",
    "SoftBank": "#fbbd09",
    "SoundCloud": "#FF8800",
    "Spotify": "#81b71a",
    "Sprint": "#fee100",
    "Stack Overflow": "#ef8236",
    "StackOverflow": "#ef8236",
    "Staples": "#cc0000",
    "Starbucks": "#00704A",
    "Stripe": "#008cdd",
    "T-Mobile": "#ea0a8e",
    "TMobile": "#ea0a8e",
    "Treehouse": "#5cb868",
    "Trello": "#256a92",
    "Trulia": "#5eab1f",
    "Twitch": "#6441a5",
    "Twitter": "#55acee",
    "Tesla": "#CC0000",
    "Tumblr": "#34526f",
    "Uber": "#1CA8C3",
    "Ubuntu": "#dd4814",
    "Verizon": "#ef1d1d",
    "Vimeo": "#44bbff",
    "Vine": "#00a478",
    "Visa": "#0157a2",
    "Vodafone": "#E90000",
    "VSCO": "#AAA94C",
    "Walmart": "#005CB0",
    "WeChat": "#93D034",
    "WhatsApp": "#34AF23",
    "WordPress": "#464646",
    "XBOX": "#9bc848",
    "Xiaomi": "#FF4A03",
    "Yahoo": "#720e9e",
    "Yandex": "#ffcc00",
    "Yelp": "#c41200",
    "Yo": "#9B59B6",
    "YouTube": "#e52d27",
    "Zendesk": "#78a300",
    "Zillow": "#0079E4",
    "ZTE": "#0A50A0"
}


# Reformateo las key del primer dic (brand_perplexity)
brand_perplexity = {k.replace("(", "-").replace(")", ""): v for k, v in brand_perplexity.items()}
# Reformateo las key del segundo dic (bc_json)
bc_json = {k.capitalize(): v for k, v in bc_json.items()}

bc_json["500px"], UIcolor["500px"]
print(len(brand_perplexity), len(bc_json), len(UIcolor))

In [ ]:
# Tengo que hacer un unico dic de ambos
brand_colors_0 = {**brand_perplexity, **bc_json, **UIcolor}
brand_colors_0['500px']

In [ ]:
brand_colors = dict(sorted(brand_colors_0.items()))
brand_colors
# print(len(brand_colors))

In [ ]:
## qwen
BRAND_COLORS = {
 '365 DataScience': '#108A99',
 '4ormat': '#fb0a2a',
 '500px': '#02adea',
 'AMD': '#ED1C24', 'AMD-2': '#000000',
 'AOL': '#00c4ff',
 'ARM': '#0084AB',
 'AT&T': '#2D96C8',
 'About-me': '#00405d', 'About-me-2': '#062f3c', 'About-me-3': '#2b82ad', 'About-me-4': '#cc7a00', 'About-me-5': '#ffcc33',
 'Acer': '#83b81a',
 'Addvocate': '#ff3322',
 'Adobe': '#ff0000', 'Adobe-1': '#FF0000', 'Adobe-2': '#fbb034', 'Adobe-3': '#ffdd00', 'Adobe-4': '#c1d82f', 'Adobe-5': '#00a4e4', 'Adobe-6': '#8a7967', 'Adobe-7': '#6a737b',
 'Aim': '#ffd900',
 'Airbnb': '#FF5A60',
 'Alibaba': '#FF7300',
 'Amazon': '#ff9900',
 'Amazon-1': '#FF9900',
 'Amazon-2': '#146eb4',
 'American Express-1': '#006FCF',
 'American Express-2': '#000000',
 'Android': '#a4c639',
 'Angies-list': '#7fbb00',
 'Aol': '#ff0b00',
 'Aol-2': '#00c4ff',
 'Apple': '#777777',
 'Apple-1': '#007AFF',
 'Apple-2': '#000000',
 'Apple-3': '#FFFFFF',
 'Arch-linux': '#1793d1',
 'Arch-linux-2': '#333333',
 'Asana': '#1d8dd5',
 'Asana-2': '#34ad00',
 'Atlassian': '#003366',
 'Audi-1': '#BB0A30',
 'Audi-2': '#000000',
 'BBC': '#333333',
 'BMW-1': '#1C69D4',
 'BMW-2': '#000000',
 'BMW-3': '#FFFFFF',
 'BT': '#084897',
 'Barnes & Noble': '#295A33',
 'Basecamp': '#6bbd6d',
 'Beats': '#FF0000',
 'Behance': '#005cff',
 'Big-cartel': '#a0ac48',
 'Big-cartel-2': '#70b29c',
 'Bing': '#ffb900',
 'Bitbucket': '#205081',
 'Bitly': '#ee6123',
 'Bitly-2': '#61b3de',
 'Blizzard': '#01B2F1',
 'Blogger': '#fc4f08',
 'Boeing': '#0039a6',
 'Booking-com': '#003580',
 'Box': '#00aeef',
 'Broadcom': '#E81231',
 'Burger King-1': '#EC1C24',
 'Burger King-2': '#FDBD10',
 'Burger King-3': '#0066B2',
 'BuzzFeed': '#EE3322',
 'Cadbury': '#472F92',
 'Canon-1': '#BF0000',
 'Canon-2': '#000000',
 'Carbonmade': '#613854',
 'Cheddar': '#ff7243',
 'China Mobile': '#1d64b8',
 'Cisco': '#11495E',
 'Coca-Cola-1': '#ED1C16',
 'Coca-Cola-2': '#000000',
 'Coca-Cola-3': '#FFFFFF',
 'Coca-cola': '#ed1c16',
 'Code-school': '#616f67',
 'Code-school-2': '#c68143',
 'Corning': '#00559B',
 'Creative-market': '#8ba753',
 'DELL': '#0085C3',
 'DHL-1': '#FFCD00',
 'DHL-2': '#D40511',
 'Delicious': '#3399ff',
 'Delicious-2': '#222222',
 'Delicious-3': '#eeeeee',
 'Dell': '#0085c3',
 'Dell-10': '#009bbb',
 'Dell-11': '#444444',
 'Dell-12': '#eeeeee',
 'Dell-2': '#7ab800',
 'Dell-3': '#f2af00',
 'Dell-4': '#dc5034',
 'Dell-5': '#ce1126',
 'Dell-6': '#b7295a',
 'Dell-7': '#6e2585',
 'Dell-8': '#71c6c1',
 'Dell-9': '#5482ab',
 'Designer News': '#2d72da',
 'Designer-news': '#2d72d9',
 'Designmoo': '#e64b50',
 'Designmoo-2': '#dbc65d',
 'Deviantart': '#4e6252',
 'Devour': '#ff0000',
 'Dewalt': '#febd17',
 'DirecTV': '#0097CD',
 'Discord-1': '#5865F2',
 'Discord-2': '#57F287',
 'Discord-3': '#FEE75C',
 'Discord-4': '#ED4245',
 'Dish Network': '#DA121D',
 'Disqus': '#2e9fff',
 'Django': '#092e20',
 'Docker-1': '#0DB7ED',
 'Docker-2': '#384D54',
 'Dribbble': '#ea4c89',
 'Dribbble-2': '#ea4c89',
 'Dribbble-3': '#8aba56',
 'Dribbble-4': '#ff8833',
 'Dribbble-5': '#00b6e3',
 'Dribbble-6': '#9ba5a8',
 'Dropbox': '#007ee5',
 'Dropbox-2': '#7b8994',
 'Dropbox-3': '#47525d',
 'Dropbox-4': '#3d464d',
 'Drupal': '#0077c0',
 'Drupal-2': '#81ceff',
 'Drupal-3': '#00598e',
 'Dunked': '#2f3238',
 'Dunked-2': '#3fc380',
 'Dyson': '#010101',
 'E4': '#6D1D7C',
 'ESPN': '#FF0033',
 'Ebay': '#e53238',
 'Ebay-2': '#0064d2',
 'Ebay-3': '#f5af02',
 'Ebay-4': '#86b817',
 'Eclass': '#cc0000',
 'Ember': '#f05e1b',
 'Engadget': '#00bdf6',
 'Envato': '#528036',
 'Ericsson': '#002561',
 'Etsy': '#eb6d20',
 'Evernote': '#7ac142',
 'Expedia': '#003050',
 'FIFA-1': '#326295',
 'FIFA-2': '#000000',
 'Fab-com': '#dd0017',
 'Facebook': '#3b5998',
 'Fairmont-1': '#A6A685',
 'Fairmont-2': '#E32119',
 'Fayerwayer': '#ff3300',
 'FedEx': '#4D148C',
 'FedEx-1': '#4D148C',
 'FedEx-2': '#FF6600',
 'Ferrari': '#1C396D',
 'Firefox': '#e66000',
 'Fitbit': '#45C2C5',
 'Flattr': '#f67c1a',
 'Flattr-2': '#338d11',
 'Flickr': '#0063dc',
 'Flickr-2': '#ff0084',
 'Flipboard': '#C10000',
 'Flipkart': '#005387',
 'Ford': '#4078C0',
 'Forrst': '#5b9a68',
 'Foursquare': '#0cbadf',
 'Foxconn': '#1E5AA0',
 'Garmin': '#007cc3',
 'General Electric': '#019DDD',
 'Geocaching': '#4a742c',
 'Getglue': '#2d75a2',
 'Gimmebar': '#f70078',
 'GitHub': '#171515',
 'GitHub-1': '#FCA326',
 'GitLab-1': '#006699',
 'Github': '#171515',
 'Gittip': '#663300',
 'Gittip-2': '#339966',
 'GoPro': '#009EE2',
 'Google': '#4285f4',
 'Google-1': '#4285F4',
 'Google-2': '#db4437',
 'Google-3': '#f4b400',
 'Google-4': '#0f9d58',
 'Google-5': '#e7e6dd',
 'Googleplus': '#dd4b39',
 'Grooveshark': '#f77f00',
 'Groupon': '#82b548',
 'HP': '#0096d6',
 'HP-1': '#0096D6',
 'HP-2': '#D7410B',
 'HTC': '#69B40F',
 'Hacker News': '#ff6600',
 'Hacker-news': '#ff6600',
 'Hellowallet': '#0085ca',
 'Heroku': '#c7c5e6',
 'Heroku-2': '#6567a5',
 'Home Depot': '#F86201',
 'Honda-1': '#CC0000',
 'Honda-2': '#000000',
 'Hootsuite': '#003366',
 'Houzz': '#73ba37',
 'Hp': '#0096d6',
 'Hsbc': '#db0011',
 'Html5': '#ec6231',
 'Huawei': '#D62D24',
 'Hulu': '#8cc83b',
 'IBM': '#003e6a',
 'IKEA': '#ffcc33',
 'IKEA-1': '#FF6600',
 'IMDb': '#f3ce13',
 'Ibm': '#003e6a',
 'Ikea': '#ffcc33',
 'Imdb': '#f3ce13',
 'Imgur': '#85bf25',
 'Instagram': '#3f729b',
 'Instagram-1': '#E4405F',
 'Instagram-2': '#405DE6',
 'Instagram-3': '#5851DB',
 'Instagram-4': '#833AB4',
 'Instagram-5': '#C13584',
 'Instagram-6': '#E1306C',
 'Instagram-7': '#FD1D1D',
 'Instagram-8': '#F56040',
 'Instagram-9': '#FCAF45',
 'Instapaper': '#1c1c1c',
 'Intel': '#0071c5',
 'Intel-1': '#0071C5',
 'Intel-2': '#00AEEF',
 'Intuit': '#365ebf',
 'JBL': '#9CB443',
 'Jawbone': '#1A1A1A',
 'KFC-1': '#F40027',
 'KFC-2': '#000000',
 'Khan Academy-1': '#242F3A',
 'Khan Academy-2': '#DDB321',
 'Kickstarter': '#76cc1e',
 'Kippt': '#e03500',
 'Kiwipay': '#00b0df',
 'Kodery': '#00af81',
 'LG': '#B20E50',
 'LG-1': '#A50034',
 'LG-2': '#C4A484',
 'LINE': '#1DCD00',
 'Lamborghini': '#0A66C2',
 'Lastfm': '#c3000d',
 'LinkedIn': '#0e76a8',
 'LinkedIn-1': '#0A3A6C',
 'Linkedin': '#007bb6',
 'Livestream': '#cf0005',
 'Lukkin': '#c90000',
 'Lumo': '#576396',
 'Lyft': '#00B4AE',
 'Lyft-1': '#FF00BF',
 'Lyft-2': '#352384',
 'MTM-1': '#113D76',
 'MTM-2': '#D6682D',
 'MTM-3': '#D8630E',
 'MTM-4': '#00A1F1',
 'Makita': '#d82028',
 'Makita-2': '#29a0b7',
 'Mastercard-1': '#FF5F00',
 'Mastercard-2': '#EB001B',
 'Mastercard-3': '#F79E1B',
 "McDonald's-1": '#FFC72C',
 "McDonald's-2": '#DA020E',
 'Meetup': '#e51937',
 'Mercedes-Benz-1': '#00ADEF',
 'Mercedes-Benz-2': '#000000',
 'Mic': '#21c1ff',
 'Microsoft': '#00a1f1',
 'Microsoft-2': '#7cbb00',
 'Microsoft-3': '#00a1f1',
 'Microsoft-4': '#ffbb00',
 'Microsoft-office': '#ea3e23',
 'Mixpanel': '#00a9d2',
 'MongoDB': '#5C92FA',
 'Motorola': '#5C92FA',
 'Mozilla': '#C34139',
 'MySQL-1': '#F29111',
 'MySQL-2': '#FFCC00',
 'NBA-1': '#1D428A',
 'NBA-2': '#C8102E',
 'NBA-3': '#000000',
 'NFL-1': '#013369',
 'NFL-2': '#D50A0A',
 'NTT DoCoMo': '#CC0033',
 'NVIDIA': '#77B900',
 'NVIDIA-1': '#76B900',
 'NVIDIA-2': '#000000',
 'National Geographic-1': '#000000',
 'National Geographic-2': '#08107B',
 'Nest': '#1EB6DC',
 'Netflix': '#b9070a',
 'Netflix-1': '#E50914',
 'Netflix-2': '#221F1F',
 'Nikon-1': '#FFD700',
 'Nikon-2': '#000000',
 'Nintendo': '#8C8C8C',
 'Nintendo-1': '#E60012',
 'Nintendo-2': '#8C8C8C',
 'Nokia': '#183693',
 'Nvidia': '#76b900',
 'Odnoklassniki': '#ed812b',
 'Olympics-1': '#0081C8',
 'Olympics-2': '#FCB131',
 'Olympics-3': '#EE334E',
 'Olympics-4': '#00A651',
 'Olympics-5': '#000000',
 'Olympus-1': '#DFB226',
 'Olympus-2': '#777777',
 'Olympus-3': '#8892BE',
 'Opera': '#cc0f16',
 'Oracle-1': '#F80000',
 'Oracle-2': '#7F7F7F',
 'PHP-1': '#4F5B93',
 'PHP-2': '#99CC99',
 'PHP-3': '#FFDE57',
 'Panasonic': '#0438C2',
 'Path': '#e41f11',
 'PayPal': '#3b7bbf',
 'PayPal-1': '#0070BA',
 'PayPal-2': '#003087',
 'PayPal-3': '#009CDE',
 'PayPal-4': '#012169',
 'Paypal-dark': '#1e477a',
 'Paypal-dark-2': '#3b7bbf',
 'Pepsi-1': '#004B93',
 'Pepsi-2': '#E32934',
 'Pepsi-3': '#FFFFFF',
 'Philips': '#0A5DD7',
 'Pinboard': '#0000e6',
 'Pinterest': '#cc2127',
 'Playstation': '#665cbe',
 'Pocket': '#ee4056',
 'Prezi': '#318bff',
 'Priceline': '#0A84C1',
 'Product Hunt': '#DA552F',
 'Pusha': '#0f71b4',
 'Python-1': '#4584B6',
 'Python-2': '#646464',
 'Python-3': '#25D366',
 'Qualcomm': '#005daa',
 'Quora': '#a82400',
 'Quote-fm': '#66ceff',
 'Rdio': '#007dc3',
 'Readability': '#9c0000',
 'Red-hat': '#cc0000',
 'Reddit': '#cee3f8',
 'Reddit-2': '#ff4500',
 'Redfin': '#A02021',
 'Resource': '#7eb400',
 'Rockpack': '#0ba6ab',
 'Roku': '#662E93',
 'Roon': '#62b0d9',
 'Rovio': '#C02227',
 'Rss': '#ee802f',
 'Salesforce': '#1798c1',
 'Salesforce-1': '#00A1E0',
 'Salesforce-2': '#16325C',
 'Salesforce-3': '#3EBAFF',
 'Samsung': '#0c4da2',
 'Samsung-1': '#1428A0',
 'Samsung-2': '#000000',
 'Shazam': '#1B87E3',
 'Shopify': '#96bf48',
 'Shopify-1': '#96BF48',
 'Shopify-2': '#5E8E3E',
 'Shopify-3': '#7AB55C',
 'Skype': '#00aff0',
 'Slack': '#6fcbdc',
 'Slack-1': '#4A154B',
 'Slack-2': '#e9a821',
 'Slack-3': '#e11664',
 'Slack-4': '#3fba91',
 'Slack-5': '#E01E5A',
 'Smashing-magazine': '#f0503a',
 'Snagajob': '#f47a20',
 'Snapchat': '#FFFC00',
 'SoftBank': '#fbbd09',
 'Softonic': '#008ace',
 'Sony-1': '#000000',
 'Sony-2': '#0070D1',
 'SoundCloud': '#FF8800',
 'Soundcloud': '#ff7700',
 'Space-box': '#f86960',
 'Spotify': '#81b71a',
 'Spotify-1': '#1DB954',
 'Spotify-2': '#191414',
 'Sprint': '#fee100',
 'Squarespace': '#121212',
 'Stack Overflow': '#ef8236',
 'StackOverflow': '#ef8236',
 'Stackoverflow': '#ef8236',
 'Staples': '#cc0000',
 'Starbucks': '#00704A',
 'Starbucks-1': '#00704A',
 'Starbucks-2': '#000000',
 'Status-chart': '#d7584f',
 'Strava': '#fc4c02',
 'Stripe': '#008cdd',
 'Stripe-1': '#635BFF',
 'Stripe-2': '#0A2540',
 'Studyblue': '#00afe1',
 'Stumbleupon': '#f74425',
 'Subway-1': '#00543C',
 'Subway-2': '#FFC72C',
 'Swarm': '#fd9627',
 'T-Mobile': '#ea0a8e',
 'T-mobile': '#ea0a8e',
 'TMobile': '#ea0a8e',
 'Technorati': '#40a800',
 'Tesla': '#CC0000',
 'Tesla-1': '#CC0000',
 'Tesla-2': '#000000',
 'The-next-web': '#ef4423',
 'TikTok-1': '#FF0050',
 'TikTok-2': '#00F2EA',
 'TikTok-3': '#000000',
 'Toyota-1': '#EB0A1E',
 'Toyota-2': '#000000',
 'Treehouse': '#5cb868',
 'Trello': '#256a92',
 'Trulia': '#5eab1f',
 'Tumblr': '#34526f',
 'Twitch': '#6441a5',
 'Twitch-1': '#9146FF',
 'Twitch-2': '#772CE8',
 'Twitch-tv': '#6441a5',
 'Twitter': '#55acee',
 'Twitter-1': '#1DA1F2',
 'Twitter-2': '#14171A',
 'Typekit': '#9aca3c',
 'Typo3': '#ff8700',
 'UPS-1': '#8A6914',
 'UPS-2': '#461A00',
 'Uber': '#1CA8C3',
 'Uber-1': '#000000',
 'Uber-2': '#FFFFFF',
 'Ubuntu': '#dd4814',
 'Ustream': '#3388ff',
 'Utorrent': '#00853f',
 'VSCO': '#AAA94C',
 'Verizon': '#ef1d1d',
 'Vimeo': '#44bbff',
 'Vine': '#00a478',
 'Virb': '#06afd8',
 'Virgin-media': '#cc0000',
 'Visa': '#0157a2',
 'Visa-1': '#1A1F71',
 'Visa-2': '#F79100',
 'Vkontakte': '#45668e',
 'Vodafone': '#E90000',
 'Volkswagen-1': '#00B1EB',
 'Volkswagen-2': '#001E50',
 'Walmart': '#005CB0',
 'WeChat': '#93D034',
 'Western Digital-5': '#003369',
 'Western Digital-6': '#FF0000',
 'WhatsApp': '#34AF23',
 'Windows': '#00bcf2',
 'Windows-phone': '#68217a',
 'Wolfram-alpha': '#f68d1e',
 'Wooga': '#5b009c',
 'WordPress': '#464646',
 'Wordpress': '#21759b',
 'Wordpress-2': '#d54e21',
 'Wordpress-3': '#464646',
 'Wordpress-com': '#1e8cbe',
 'Wunderlist': '#2b88d9',
 'XBOX': '#9bc848',
 'Xbox': '#52b043',
 'Xiaomi': '#FF4A03',
 'Xing': '#126567',
 'Yahoo': '#720e9e',
 'Yandex': '#ffcc00',
 'Yelp': '#c41200',
 'Yo': '#9B59B6',
 'YouTube': '#e52d27',
 'YouTube-1': '#FF0000',
 'YouTube-2': '#282828',
 'Youtube': '#cd201f',
 'ZTE': '#0A50A0',
 'Zalongo': '#5498dc',
 'Zendesk': '#78a300',
 'Zerply': '#9dcc7a',
 'Zillow': '#0079E4',
 'Zoom-1': '#2D8CFF',
 'Zoom-2': '#9B9B9B',
 'Zootool': '#5e8b1d',
 'Zopim': '#ff9d3b',
 'about.me': '#00405d',
 'bitly': '#ee6123',
 'eBay': '#e53238',
 'iHeartRadio': '#BC1C10'}

In [ ]:
    [map dk_updateKey:@"500px" hexColor:@"02adea"];
    
    [map dk_updateKey:@"about.me" hexColor:@"00405d"];
    [map dk_updateKey:@"Acer" hexColor:@"83b81a"];
    [map dk_updateKey:@"Adobe" hexColor:@"ff0000"];
    [map dk_updateKey:@"Airbnb" hexColor:@"FF5A60"];
    [map dk_updateKey:@"Alibaba" hexColor: @"FF7300"];
    [map dk_updateKey:@"Amazon" hexColor:@"ff9900"];
    [map dk_updateKey:@"Android" hexColor:@"a4c639"];
    [map dk_updateKey:@"Apple" hexColor:@"777777"];
    [map dk_updateKey:@"AOL" hexColor:@"00c4ff"];
    [map dk_updateKey:@"ARM" hexColor:@"0084AB"];
    [map dk_updateKey:@"Asana" hexColor:@"1d8dd5"];
    [map dk_updateKey:@"AT&T" hexColor:@"2D96C8"];
    [map dk_updateKey:@"Atlassian" hexColor:@"003366"];
    
    [map dk_updateKey:@"Barnes & Noble" hexColor:@"295A33"];
    [map dk_updateKey:@"BBC" hexColor:@"333333"];
    [map dk_updateKey:@"Beats" hexColor:@"FF0000"];
    [map dk_updateKey:@"Behance" hexColor:@"005cff"];
    [map dk_updateKey:@"Bing" hexColor:@"ffb900"];
    [map dk_updateKey:@"bitly" hexColor:@"ee6123"];
    [map dk_updateKey:@"Blizzard" hexColor:@"01B2F1"];
    [map dk_updateKey:@"Blogger" hexColor:@"fc4f08"];
    [map dk_updateKey:@"Box" hexColor:@"00aeef"];
    [map dk_updateKey:@"Broadcom" hexColor:@"E81231"];
    [map dk_updateKey:@"BT" hexColor:@"084897"];
    [map dk_updateKey:@"BuzzFeed" hexColor:@"EE3322"];
    [map dk_updateKey:@"China Mobile" hexColor:@"1d64b8"];
    
    [map dk_updateKey:@"Cisco" hexColor:@"11495E"];
    [map dk_updateKey:@"Corning" hexColor:@"00559B"];
    
    [map dk_updateKey:@"Dell" hexColor:@"0085c3"];
    [map dk_updateKey:@"Designer News" hexColor:@"2d72da"];
    [map dk_updateKey:@"DirecTV" hexColor:@"0097CD"];
    [map dk_updateKey:@"Dish Network" hexColor:@"DA121D"];
    [map dk_updateKey:@"Dribbble" hexColor:@"ea4c89"];
    [map dk_updateKey:@"Dropbox" hexColor:@"007ee5"];
    [map dk_updateKey:@"Dyson" hexColor:@"010101"];
    
    [map dk_updateKey:@"eBay" hexColor:@"e53238"];
    [map dk_updateKey:@"Ember" hexColor:@"f05e1b"];
    [map dk_updateKey:@"Engadget" hexColor:@"00bdf6"];
    [map dk_updateKey:@"Ericsson" hexColor:@"002561"];
    [map dk_updateKey:@"Etsy" hexColor:@"eb6d20"];
    [map dk_updateKey:@"Evernote" hexColor:@"7ac142"];
    [map dk_updateKey:@"Expedia" hexColor:@"003050"];
    
    [map dk_updateKey:@"Facebook" hexColor:@"3b5998"];
    [map dk_updateKey:@"FedEx" hexColor:@"4D148C"];
    [map dk_updateKey:@"Firefox" hexColor:@"e66000"];
    [map dk_updateKey:@"Fitbit" hexColor:@"45C2C5"];
    [map dk_updateKey:@"Flickr" hexColor:@"0063dc"];
    [map dk_updateKey:@"Flipboard" hexColor:@"C10000"];
    [map dk_updateKey:@"Flipkart" hexColor:@"005387"];
    [map dk_updateKey:@"Foxconn" hexColor:@"1E5AA0"];
    [map dk_updateKey:@"Foursquare" hexColor:@"0cbadf"];

    [map dk_updateKey:@"General Electric" hexColor:@"019DDD"];
    [map dk_updateKey:@"GitHub" hexColor:@"171515"];
    [map dk_updateKey:@"Google" hexColor:@"4285f4"];
    [map dk_updateKey:@"GoPro" hexColor:@"009EE2"];
    
    [map dk_updateKey:@"Hacker News" hexColor:@"ff6600"];
    [map dk_updateKey:@"HP" hexColor:@"0096d6"];
    [map dk_updateKey:@"Home Depot" hexColor:@"F86201"];
    [map dk_updateKey:@"HTC" hexColor:@"69B40F"];
    [map dk_updateKey:@"Huawei" hexColor:@"D62D24"];
    [map dk_updateKey:@"Hulu" hexColor:@"8cc83b"];
    
    [map dk_updateKey:@"IBM" hexColor:@"003e6a"];
    [map dk_updateKey:@"iHeartRadio" hexColor:@"BC1C10"];
    [map dk_updateKey:@"IKEA" hexColor:@"ffcc33"];
    [map dk_updateKey:@"IMDb" hexColor:@"f3ce13"];
    [map dk_updateKey:@"Instagram" hexColor:@"3f729b"];
    [map dk_updateKey:@"Intel" hexColor:@"0071c5"];
    [map dk_updateKey:@"Intuit" hexColor:@"365ebf"];
    
    [map dk_updateKey:@"Jawbone" hexColor:@"1A1A1A"];
    
    [map dk_updateKey:@"Kickstarter" hexColor:@"76cc1e"];
    
    [map dk_updateKey:@"LG" hexColor:@"B20E50"];
    [map dk_updateKey:@"LinkedIn" hexColor:@"0e76a8"];
    [map dk_updateKey:@"LINE" hexColor:@"1DCD00"];
    [map dk_updateKey:@"Lyft" hexColor:@"00B4AE"];
    [map dk_updateKey:@"Microsoft" hexColor:@"00a1f1"];
    [map dk_updateKey:@"Mixpanel" hexColor:@"00a9d2"];
    [map dk_updateKey:@"Motorola" hexColor:@"5C92FA"];
    [map dk_updateKey:@"Mozilla" hexColor:@"C34139"];
    
    [map dk_updateKey:@"Nest" hexColor:@"1EB6DC"];
    [map dk_updateKey:@"Netflix" hexColor:@"b9070a"];
    [map dk_updateKey:@"Nintendo" hexColor:@"8C8C8C"];
    [map dk_updateKey:@"Nokia" hexColor:@"183693"];
    [map dk_updateKey:@"NTT DoCoMo" hexColor:@"CC0033"];
    [map dk_updateKey:@"NVIDIA" hexColor:@"77B900"];
    
    [map dk_updateKey:@"Opera" hexColor:@"cc0f16"];
 
    [map dk_updateKey:@"Panasonic" hexColor:@"0438C2"];
    [map dk_updateKey:@"PayPal" hexColor:@"3b7bbf"];
    [map dk_updateKey:@"Pinterest" hexColor:@"cc2127"];
    [map dk_updateKey:@"Philips" hexColor:@"0A5DD7"];
    [map dk_updateKey:@"Priceline" hexColor:@"0A84C1"];
    [map dk_updateKey:@"Product Hunt" hexColor:@"DA552F"];
    
    [map dk_updateKey:@"Qualcomm" hexColor:@"005daa"];
    
    [map dk_updateKey:@"Reddit" hexColor:@"cee3f8" isLightColor:YES];
    [map dk_updateKey:@"Redfin" hexColor:@"A02021"];
    [map dk_updateKey:@"Roku" hexColor:@"662E93"];
    [map dk_updateKey:@"Rovio" hexColor:@"C02227"];
    [map dk_updateKey:@"Rdio" hexColor:@"007dc3"];
    
    [map dk_updateKey:@"Salesforce" hexColor:@"1798c1"];
    [map dk_updateKey:@"Samsung" hexColor:@"0c4da2"];
    [map dk_updateKey:@"Snapchat" hexColor:@"FFFC00" isLightColor:YES];
    [map dk_updateKey:@"Shazam" hexColor:@"1B87E3"];
    [map dk_updateKey:@"Skype" hexColor:@"00aff0"];
    [map dk_updateKey:@"SoftBank" hexColor:@"fbbd09"];
    [map dk_updateKey:@"SoundCloud" hexColor:@"FF8800"];
    [map dk_updateKey:@"Spotify" hexColor:@"81b71a"];
    [map dk_updateKey:@"Sprint" hexColor:@"fee100"];
    [map dk_updateKey:@"Stack Overflow" hexColor:@"ef8236"];
    [map dk_updateKey:@"StackOverflow" hexColor:@"ef8236"];
    [map dk_updateKey:@"Staples" hexColor:@"cc0000"];
    [map dk_updateKey:@"Starbucks" hexColor:@"00704A"];
    [map dk_updateKey:@"Stripe" hexColor:@"008cdd"];
    
    [map dk_updateKey:@"T-Mobile" hexColor:@"ea0a8e"];
    [map dk_updateKey:@"TMobile" hexColor:@"ea0a8e"];
    [map dk_updateKey:@"Treehouse" hexColor:@"5cb868"];
    [map dk_updateKey:@"Trello" hexColor:@"256a92"];
    [map dk_updateKey:@"Trulia" hexColor:@"5eab1f"];
    [map dk_updateKey:@"Twitch" hexColor:@"6441a5"];
    [map dk_updateKey:@"Twitter" hexColor:@"55acee"];
    [map dk_updateKey:@"Tesla" hexColor:@"CC0000"];
    [map dk_updateKey:@"Tumblr" hexColor:@"34526f"];
    
    [map dk_updateKey:@"Uber" hexColor:@"1CA8C3"];
    [map dk_updateKey:@"Ubuntu" hexColor:@"dd4814"];
    
    [map dk_updateKey:@"Verizon" hexColor:@"ef1d1d"];
    [map dk_updateKey:@"Vimeo" hexColor:@"44bbff"];
    [map dk_updateKey:@"Vine" hexColor:@"00a478"];
    [map dk_updateKey:@"Visa" hexColor:@"0157a2"];
    [map dk_updateKey:@"Vodafone" hexColor:@"E90000"];
    [map dk_updateKey:@"VSCO" hexColor:@"AAA94C"];
    
    [map dk_updateKey:@"Walmart" hexColor:@"005CB0"];
    [map dk_updateKey:@"WeChat" hexColor:@"93D034"];
    [map dk_updateKey:@"WhatsApp" hexColor:@"34AF23"];
    [map dk_updateKey:@"WordPress" hexColor:@"464646"];
    
    [map dk_updateKey:@"XBOX" hexColor:@"9bc848"];
    [map dk_updateKey:@"Xiaomi" hexColor:@"FF4A03"];
    
    [map dk_updateKey:@"Yahoo" hexColor:@"720e9e"];
    [map dk_updateKey:@"Yandex" hexColor:@"ffcc00"];
    [map dk_updateKey:@"Yelp" hexColor:@"c41200"];
    [map dk_updateKey:@"Yo" hexColor:@"9B59B6"];
    [map dk_updateKey:@"YouTube" hexColor:@"e52d27"];
    
    [map dk_updateKey:@"Zendesk" hexColor:@"78a300"];
    [map dk_updateKey:@"Zillow" hexColor:@"0079E4"];
    [map dk_updateKey:@"ZTE" hexColor:@"0A50A0"];
    